In [6]:
import pandas as pd
import numpy as np
import os

In [7]:
file_path = '2020_LER_20201102_testset_V04.xlsx'

# file_path에 따라 변수 결정 (N형만 생각)

if file_path == '2020_LER_20200804_V006.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = 100
    num_total = num_in_cycle * num_of_cycle
    num_train = 75
    num_val = 10
    num_test = 15
    x_cols = "B:G"
    y_cols = "H:P"
    header = 0

elif file_path == '2020_LER_20201008_V008.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = 127
    num_total = num_in_cycle * num_of_cycle
    num_train = 75
    num_val = 10
    num_test = 0
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2020_LER_20200922_testset.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = [232, 289, 277, 253, 255]
    num_of_cycle = 5
    num_total = sum(num_in_cycle)
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2020_LER_20201021_testset.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = [232, 289, 277, 253, 255, 250, 250, 250]
    num_of_cycle = 8
    num_total = sum(num_in_cycle)
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2020_RDFWFV_20201029_V08.xlsx':
    num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
    num_output = 6
    num_in_cycle_train = 50
    num_in_cycle_test = 250
    num_of_cycle_train = [50, 50, 100]
    num_of_cycle_test = [2, 0, 2]
    num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
    num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
    header = 0
    
elif file_path == '2020_RDFWFV_20201101_V09.xlsx':
    num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
    num_output = 6
    num_in_cycle_train = 50
    num_in_cycle_test = 250
    num_of_cycle_train = [50, 50, 100]
    num_of_cycle_test = [2, 0, 4]
    num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
    num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
    header = 0
    
elif file_path == '2020_LER_20201102_testset_V04.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = [250]*10
    num_of_cycle = 10
    num_total = sum(num_in_cycle)
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == 'constant current Test set_V00':
    num_input = 5
    num_output = 6
    num_in_cycle = [250]*10
    num_of_cycle = 10
    num_total = sum(num_in_cycle)
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2021_RDFWFV_20210107.xlsx':
    num_input = 6
    one_hot = 3
    num_output = 8
    num_in_cycle = 250
    num_of_cycle = [5, 5, 6]
    num_total = 250*sum(num_of_cycle)
    x_cols = "A:F"
    y_cols = "G:N"
    header = 0

# 2020_LER_20200804_V006

In [75]:
# file_path version : 2020_LER_20200804_V006.xlsx

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=2*num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=2*num_total+1, header=header)

# one-hot encoding
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing
# 1. N, P to 10, 01 (one-hot encoding)
data_x =pd.get_dummies(data_x, columns=['PNMOS'], dtype=float)

# 2. Remove unrequired column ( Wfin [nm], alpha )
# X: Wfin, alpha
data_x = data_x.drop('Wfin [nm]', axis=1)
data_x = data_x.drop('alpha', axis=1)

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)

# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
#     print(2*i*num_in_cycle, 2*i*num_in_cycle+1)
    X_per_cycle[i] = data_x[2*i*num_in_cycle:2*i*num_in_cycle+1].values
#     print(X_per_cycle[i])

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)

# Y_all
for i in range(num_of_cycle):
#     print(i*num_in_cycle,(i+1)*num_in_cycle)
    Y_all[i*num_in_cycle:i*num_in_cycle+num_in_cycle] = data_y[2*i*num_in_cycle:2*i*num_in_cycle + num_in_cycle].values
#     print(Y_all[i*num_in_cycle])

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

============ Data load =============
X data shape:  (5000, 5) X per cycle data shape: (100, 5)
Y data shape:  (5000, 6) Y per cycle data shape: (100, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [79]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

ㅔ갸ㅜ
for i in range(X_per_cycle.shape[0]):
    print(i)
    print(X_per_cycle[i])

(5000, 5) (5000, 6) (100, 5) (100, 6)
0
[  0.19316255  61.00227254 110.61789695   1.           0.        ]
1
[  0.71479321  30.46485839 164.03098759   1.           0.        ]
2
[  0.36639042  45.71598335 128.60254701   1.           0.        ]
3
[ 0.19156989 92.07622363 49.23479694  1.          0.        ]
4
[  0.64566003  28.77580661 181.68354021   1.           0.        ]
5
[  0.24976869  49.66628494 160.81107576   1.           0.        ]
6
[  0.58558021  33.64448087 197.99245715   1.           0.        ]
7
[ 0.6574629  45.59676738 70.67648175  1.          0.        ]
8
[ 0.13748646 47.26343432 23.46686409  1.          0.        ]
9
[  0.76475293  92.24596085 177.48935442   1.           0.        ]
10
[  0.29626603  81.48909977 179.64037297   1.           0.        ]
11
[ 0.6236913  87.47656503 97.32992467  1.          0.        ]
12
[  0.46475666  46.051489   122.05853503   1.           0.        ]
13
[  0.57950422  29.18162993 132.20992165   1.           0.        ]
14
[ 0.19323

# 2020_LER_20201008_V008

In [62]:
# file_path version : 2020_LER_20201008_V006.xlsx

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
#     print(i*num_in_cycle, i*num_in_cycle+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle[i])

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))


# Y_all
for i in range(num_of_cycle):
#     print(i*num_in_cycle,(i+1)*num_in_cycle)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all[i*num_in_cycle])

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

          amp.     corr.x      corr.y
0     0.193163  61.002273  110.617897
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
6345       NaN        NaN         NaN
6346       NaN        NaN         NaN
6347       NaN        NaN         NaN
6348       NaN        NaN         NaN
6349       NaN        NaN         NaN

[6350 rows x 3 columns]
============ Data load =============
X data shape:  (6350, 5) X per cycle data shape: (127, 5)
Y data shape:  (6350, 6) Y per cycle data shape: (127, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [64]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(X_per_cycle[i])

(6350, 5) (6350, 6) (127, 5) (127, 6)
[  0.19316255  61.00227254 110.61789695   1.           0.        ]
[  0.71479321  30.46485839 164.03098759   1.           0.        ]
[  0.36639042  45.71598335 128.60254701   1.           0.        ]
[ 0.19156989 92.07622363 49.23479694  1.          0.        ]
[  0.64566003  28.77580661 181.68354021   1.           0.        ]
[  0.24976869  49.66628494 160.81107576   1.           0.        ]
[  0.58558021  33.64448087 197.99245715   1.           0.        ]
[ 0.6574629  45.59676738 70.67648175  1.          0.        ]
[ 0.13748646 47.26343432 23.46686409  1.          0.        ]
[  0.76475293  92.24596085 177.48935442   1.           0.        ]
[  0.29626603  81.48909977 179.64037297   1.           0.        ]
[ 0.6236913  87.47656503 97.32992467  1.          0.        ]
[  0.46475666  46.051489   122.05853503   1.           0.        ]
[  0.57950422  29.18162993 132.20992165   1.           0.        ]
[ 0.19323371 43.90356688 60.43297631  1.    

IndexError: index 127 is out of bounds for axis 0 with size 127

# 2020_LER_0922_testset

In [98]:
# file_path version : 2020_LER_20201008_V006.xlsx

file_path = '2020_LER_20200922_testset.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

idx = 0
add = 0
for i in range(num_of_cycle):
    add = num_in_cycle[i]
            
    X_per_cycle[i] = X_all[idx:idx+1]
    Y_per_cycle[i] = np.mean(Y_all[idx:idx+add], axis=0)
            
    temp = X_per_cycle[i].reshape(1, num_input-2)
            
    X_all[idx:idx+add] = np.repeat(temp, add, axis=0)
    idx += add
    
# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
1301       NaN        NaN         NaN
1302       NaN        NaN         NaN
1303       NaN        NaN         NaN
1304       NaN        NaN         NaN
1305       NaN        NaN         NaN

[1306 rows x 3 columns]
============ Data load =============
X data shape:  (1306, 5) X per cycle data shape: (5, 5)
Y data shape:  (1306, 6) Y per cycle data shape: (5, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [99]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

(1306, 5) (1306, 6) (5, 5) (5, 6)
[1.9580e-12 4.2420e-05 1.6180e-05 3.4300e-01 3.5300e-01 6.1501e+01]
[6.1350e-13 3.6300e-05 1.4500e-05 3.7000e-01 3.7700e-01 6.0804e+01]
[7.437e-13 3.182e-05 1.391e-05 3.700e-01 3.740e-01 6.120e+01]
[1.4790e-12 4.1000e-05 1.5990e-05 3.5200e-01 3.5800e-01 6.1429e+01]
[1.9260e-12 4.3930e-05 1.7650e-05 3.4400e-01 3.5300e-01 6.1569e+01]
[1.1990e-12 3.6920e-05 1.4540e-05 3.5600e-01 3.6300e-01 6.1272e+01]
[1.6050e-12 3.6600e-05 1.3990e-05 3.4900e-01 3.5500e-01 6.1348e+01]
[1.5290e-12 4.0250e-05 1.5810e-05 3.5000e-01 3.5700e-01 6.1406e+01]
[2.2580e-12 3.9540e-05 1.6170e-05 3.4200e-01 3.5100e-01 6.1827e+01]
[1.9230e-12 4.2050e-05 1.5870e-05 3.4500e-01 3.5300e-01 6.1627e+01]
[1.1800e-12 4.0380e-05 1.7000e-05 3.5500e-01 3.6300e-01 6.1232e+01]
[2.461e-12 4.101e-05 1.478e-05 3.390e-01 3.490e-01 6.173e+01]
[3.8970e-12 4.2100e-05 1.6900e-05 3.3100e-01 3.4100e-01 6.2348e+01]
[2.6110e-12 4.3680e-05 1.6380e-05 3.3700e-01 3.4800e-01 6.1783e+01]
[2.5380e-12 4.2210e-05 1.6

[1.9460e-12 4.2370e-05 1.6740e-05 3.4400e-01 3.5200e-01 6.1528e+01]
[2.8210e-12 4.3190e-05 1.6510e-05 3.3700e-01 3.4500e-01 6.1912e+01]
[1.2100e-12 4.0100e-05 1.5840e-05 3.5500e-01 3.6100e-01 6.1229e+01]
[1.1900e-12 3.9310e-05 1.5080e-05 3.5500e-01 3.6200e-01 6.1208e+01]
[1.5130e-12 3.9020e-05 1.6100e-05 3.5100e-01 3.5700e-01 6.1536e+01]
[3.7450e-12 4.3600e-05 1.7810e-05 3.3200e-01 3.4100e-01 6.2351e+01]
[4.030e-12 4.368e-05 1.739e-05 3.310e-01 3.410e-01 6.256e+01]
[3.539e-12 4.400e-05 1.725e-05 3.320e-01 3.430e-01 6.221e+01]
[1.4000e-12 4.0690e-05 1.4720e-05 3.5100e-01 3.5900e-01 6.1247e+01]
[2.2210e-12 4.1420e-05 1.7000e-05 3.4300e-01 3.4900e-01 6.1841e+01]
[2.000e-12 4.265e-05 1.691e-05 3.430e-01 3.510e-01 6.155e+01]
[1.9700e-12 4.1760e-05 1.5860e-05 3.4500e-01 3.5200e-01 6.1595e+01]
[2.064e-12 4.240e-05 1.687e-05 3.440e-01 3.510e-01 6.173e+01]
[2.2890e-12 4.0890e-05 1.5820e-05 3.4000e-01 3.4800e-01 6.1602e+01]
[1.2440e-12 3.4430e-05 1.4750e-05 3.5500e-01 3.6100e-01 6.1226e+01]
[1.9

 # 2020_LER_1021_testset

In [4]:
# file_path version : 2020_LER_20201008_V006.xlsx

print(num_total)
file_path = '2020_LER_20201021_testset.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Generated Datas', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated Datas', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

idx = 0
add = 0
for i in range(num_of_cycle):
    add = num_in_cycle[i]
            
    X_per_cycle[i] = X_all[idx:idx+1]
    Y_per_cycle[i] = np.mean(Y_all[idx:idx+add], axis=0)
            
    temp = X_per_cycle[i].reshape(1, num_input-2)
            
    X_all[idx:idx+add] = np.repeat(temp, add, axis=0)
    idx += add
    
# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

2500
          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
2051       NaN        NaN         NaN
2052       NaN        NaN         NaN
2053       NaN        NaN         NaN
2054       NaN        NaN         NaN
2055       NaN        NaN         NaN

[2056 rows x 3 columns]


ValueError: could not broadcast input array from shape (250,3) into shape (56,3)

In [19]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)


#print(Y_all[])

for i in range(1306, 1556):
    print(Y_all[i])
#print(Y_all[1306:1556])

total_sum = 0
for i in [232, 289, 277, 253, 255, 250, 250, 250]:
    total_sum += i
    print(total_sum)
    
# for i in range(Y_all.shape[0]):
#     if 
#     print(Y_all[i])

(2056, 5) (2056, 6) (8, 5) (8, 6)
[2.0210e-12 4.2070e-05 1.6100e-05 3.4300e-01 3.5000e-01 6.1526e+01]
[1.7980e-12 4.2010e-05 1.6180e-05 3.4500e-01 3.5300e-01 6.1443e+01]
[1.7630e-12 4.2270e-05 1.6260e-05 3.4600e-01 3.5300e-01 6.1426e+01]
[2.1660e-12 4.2790e-05 1.6390e-05 3.4100e-01 3.4900e-01 6.1548e+01]
[2.4170e-12 4.3730e-05 1.7310e-05 3.4000e-01 3.4700e-01 6.1766e+01]
[2.1470e-12 4.2130e-05 1.6300e-05 3.4200e-01 3.4900e-01 6.1609e+01]
[2.3030e-12 4.2860e-05 1.6450e-05 3.4100e-01 3.4800e-01 6.1703e+01]
[1.9840e-12 4.2930e-05 1.6300e-05 3.4300e-01 3.5100e-01 6.1494e+01]
[2.6190e-12 4.2340e-05 1.6540e-05 3.3800e-01 3.4600e-01 6.1756e+01]
[1.8640e-12 4.2090e-05 1.5910e-05 3.4400e-01 3.5200e-01 6.1399e+01]
[2.0250e-12 4.2800e-05 1.6540e-05 3.4300e-01 3.5000e-01 6.1619e+01]
[2.3020e-12 4.2440e-05 1.6790e-05 3.4100e-01 3.4800e-01 6.1707e+01]
[1.7970e-12 4.2450e-05 1.6290e-05 3.4500e-01 3.5300e-01 6.1464e+01]
[2.2230e-12 4.2590e-05 1.6600e-05 3.4100e-01 3.4800e-01 6.1671e+01]
[2.3430e-12 4.

# 2020_RDFWFV_20201029_V08

### 1. RDF ( train + test )

In [70]:
# file_path == '2020_RDFWFV_20201029_V08.xlsx':
#     num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
#     num_output = 6
#     num_in_cycle_train = 50
#     num_in_cycle_test = 250
#     num_of_cycle_train = [50, 50, 100]
#     num_of_cycle_test = [2, 0, 2]
#     num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
#     num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
#     header = 0

file_path = '2020_RDFWFV_20201029_V08.xlsx'

#### TRAIN

## 1.read from excel
train_data_x = pd.read_excel('../'+file_path, sheet_name='RDF data', usecols="C:E", nrows=num_total_train[0]+1, header=header)
train_data_y = pd.read_excel('../'+file_path, sheet_name='RDF data', usecols="F:M", nrows=num_total_train[0]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_train , Y_all_train = np.zeros((num_total_train[0], num_input[0]-2)), np.zeros((num_total_train[0], num_output))
X_per_cycle_train, Y_per_cycle_train = np.zeros((num_of_cycle_train[0], num_input[0]-2)), np.zeros((num_of_cycle_train[0], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
train_data_y = train_data_y.drop('Idlo', axis=1)
train_data_y = train_data_y.drop('Idhi', axis=1)

## 4. X_per_cycle
for i in range(num_of_cycle_train[0]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle_train[i] = train_data_x[i*num_in_cycle_train:i*num_in_cycle_train+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all_train = np.repeat(X_per_cycle_train,num_in_cycle_train,axis=0)


## 6. 인위적으로 one-hot 넣어주기
X_all_train = np.hstack((X_all_train, np.ones((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.ones((X_per_cycle_train.shape[0], 1))))

X_all_train = np.hstack((X_all_train, np.zeros((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.zeros((X_per_cycle_train.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle_train[0]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train] = train_data_y[i*num_in_cycle_train:(i+1)*num_in_cycle_train].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle_train[0]):
    Y_per_cycle_train[i] = np.mean(Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train],axis=0)
#     print(i, Y_per_cycle_train[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all_train.shape, "X per cycle data shape:", X_per_cycle_train.shape)
print("Y data shape: ", Y_all_train.shape, "Y per cycle data shape:", Y_per_cycle_train.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_train)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_train)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all_train)
data_train.append(Y_all_train)
data_train.append(X_per_cycle_train)
data_train.append(Y_per_cycle_train)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdf_train'+file_path, data_train)
print()

#### TEST

## 1.read from excel
test_data_x = pd.read_excel('../'+file_path, sheet_name='RDF testdata', usecols="B:D", nrows=num_total_test[0]+1, header=header)
test_data_y = pd.read_excel('../'+file_path, sheet_name='RDF testdata', usecols="E:L", nrows=num_total_test[0]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_test , Y_all_test = np.zeros((num_total_test[0], num_input[0]-2)), np.zeros((num_total_test[0], num_output))
X_per_cycle_test, Y_per_cycle_test = np.zeros((num_of_cycle_test[0], num_input[0]-2)), np.zeros((num_of_cycle_test[0], num_output))

## 3. DATA_X DATA_Y preprocessing
test_data_y = test_data_y.drop('Idlo', axis=1)
test_data_y = test_data_y.drop('Idhi', axis=1)

## 4. X_per_cycle, Y_all
for i in range(num_of_cycle_test[0]):
    X_per_cycle_test[i] = test_data_x[i*num_in_cycle_test:i*num_in_cycle_test+1].values
    
Y_all_test = test_data_y.values    

## 5. X_all, Y_per_cycle
for i in range(num_of_cycle_test[0]):
#     print(i*num_in_cycle_test, (i+1)*num_in_cycle_test)
#     print(X_per_cycle_test[i])
#     print(np.reshape(X_per_cycle_test[i], (1,3)).shape)
    X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test] = np.repeat(np.reshape(X_per_cycle_test[i], (1,3)), num_in_cycle_test, axis=0)
#     print(X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test])

    Y_per_cycle_test[i] = np.mean(Y_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test], axis=0)

## 6. 인위적으로 one-hot 넣어주기
X_all_test = np.hstack((X_all_test, np.ones((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.ones((X_per_cycle_test.shape[0], 1))))

X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))    
    
# for i in range(X_all_test.shape[0]):
#     print(X_all_test[i])
#     print(Y_all_test[i])
    
    
print("============ TEST Data load =============")
print()
print("X data shape: ", X_all_test.shape, "X per cycle data shape:", X_per_cycle_test.shape)
print("Y data shape: ", Y_all_test.shape, "Y per cycle data shape:", Y_per_cycle_test.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_test)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_test)))  
print()
print("============ TRAIN Data save =============")
print()
data_test = []
data_test.append(X_all_test)
data_test.append(Y_all_test)
data_test.append(X_per_cycle_test)
data_test.append(Y_per_cycle_test)
    
data_test = np.array(data_test)
print("saved data shape: ", data_test.shape)
np.save('./rdf_test'+file_path, data_test)
print()

============ TRAIN Data load =============

X data shape:  (2500, 5) X per cycle data shape: (50, 5)
Y data shape:  (2500, 6) Y per cycle data shape: (50, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============

saved data shape:  (4,)

============ TEST Data load =============

X data shape:  (500, 5) X per cycle data shape: (2, 5)
Y data shape:  (500, 6) Y per cycle data shape: (2, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============

saved data shape:  (4,)



In [80]:
##### train
file_path = 'rdf_train2020_RDFWFV_20201029_V08.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

# test    
test_file_path = 'rdf_test2020_RDFWFV_20201029_V08.xlsx'

test_data = np.load(test_file_path+'.npy', allow_pickle=True)

test_X_all, test_Y_all, test_X_per_cycle, test_Y_per_cycle = test_data[0], test_data[1], test_data[2], test_data[3]

print(test_data[0].shape, test_data[1].shape, test_data[2].shape, test_data[3].shape)

for i in range(test_Y_all.shape[0]):
    print(test_Y_all[i])

(2500, 5) (2500, 6) (50, 5) (50, 6)
[2.1610e-12 3.4800e-01 3.4100e-01 7.3110e-06 2.1470e-05 6.1024e+01]
[2.1160e-12 3.4800e-01 3.4200e-01 7.7550e-06 2.3090e-05 6.1121e+01]
[2.1250e-12 3.4800e-01 3.4200e-01 7.7380e-06 2.1880e-05 6.1073e+01]
[2.318e-12 3.460e-01 3.400e-01 8.019e-06 2.576e-05 6.111e+01]
[2.1720e-12 3.4700e-01 3.4100e-01 7.8290e-06 2.1890e-05 6.0965e+01]
[2.0670e-12 3.4900e-01 3.4200e-01 7.3110e-06 2.2150e-05 6.0968e+01]
[2.1180e-12 3.4900e-01 3.4200e-01 7.5880e-06 2.2200e-05 6.1032e+01]
[2.0730e-12 3.4900e-01 3.4300e-01 7.6010e-06 2.0940e-05 6.1044e+01]
[2.1320e-12 3.4800e-01 3.4200e-01 7.9540e-06 2.1880e-05 6.1028e+01]
[2.2110e-12 3.4700e-01 3.4100e-01 7.4870e-06 2.2470e-05 6.1097e+01]
[2.031e-12 3.490e-01 3.430e-01 7.503e-06 2.091e-05 6.102e+01]
[2.2500e-12 3.4700e-01 3.4000e-01 7.9310e-06 2.3910e-05 6.1036e+01]
[1.978e-12 3.510e-01 3.440e-01 7.067e-06 1.963e-05 6.100e+01]
[2.1100e-12 3.4800e-01 3.4200e-01 7.4210e-06 2.2120e-05 6.0992e+01]
[2.0720e-12 3.4900e-01 3.4200e

[3.3200e-12 3.4100e-01 3.3200e-01 1.7060e-05 4.4550e-05 6.1966e+01]
[3.327e-12 3.410e-01 3.320e-01 1.735e-05 4.442e-05 6.202e+01]
[3.4250e-12 3.4000e-01 3.3100e-01 1.7570e-05 4.5870e-05 6.1902e+01]
[3.3590e-12 3.4000e-01 3.3200e-01 1.7350e-05 4.4300e-05 6.1953e+01]
[3.1660e-12 3.4100e-01 3.3300e-01 1.6790e-05 4.4050e-05 6.1941e+01]
[3.3140e-12 3.4000e-01 3.3200e-01 1.8400e-05 4.6890e-05 6.1896e+01]
[3.366e-12 3.400e-01 3.320e-01 1.739e-05 4.511e-05 6.193e+01]
[3.5010e-12 3.4000e-01 3.3100e-01 1.7260e-05 4.4660e-05 6.1974e+01]
[3.3970e-12 3.4000e-01 3.3200e-01 1.6940e-05 4.3720e-05 6.1895e+01]
[3.7220e-12 3.3900e-01 3.2900e-01 1.8860e-05 4.7810e-05 6.1977e+01]
[3.4180e-12 3.4000e-01 3.3200e-01 1.8150e-05 4.5180e-05 6.2025e+01]
[3.1700e-12 3.4200e-01 3.3300e-01 1.6750e-05 4.3040e-05 6.1921e+01]
[3.3730e-12 3.4000e-01 3.3200e-01 1.7560e-05 4.5100e-05 6.1985e+01]
[3.3930e-12 3.4000e-01 3.3200e-01 1.8150e-05 4.5880e-05 6.2023e+01]
[3.4750e-12 3.3900e-01 3.3100e-01 1.7540e-05 4.5550e-05 6.19

[4.1530e-12 3.3900e-01 3.2900e-01 2.5170e-05 5.4710e-05 6.2351e+01]
[3.6690e-12 3.4200e-01 3.3200e-01 2.1700e-05 4.8680e-05 6.2297e+01]
[4.1330e-12 3.4000e-01 3.2900e-01 2.4050e-05 5.3750e-05 6.2372e+01]
[3.9130e-12 3.4100e-01 3.3000e-01 2.2170e-05 5.0880e-05 6.2325e+01]
[3.968e-12 3.390e-01 3.300e-01 2.291e-05 5.114e-05 6.235e+01]
[3.8820e-12 3.4100e-01 3.3100e-01 2.3470e-05 5.2190e-05 6.2392e+01]
[3.8590e-12 3.4000e-01 3.3100e-01 2.2480e-05 4.9320e-05 6.2294e+01]
[4.0200e-12 3.4100e-01 3.2900e-01 2.3710e-05 5.3100e-05 6.2279e+01]
[3.9080e-12 3.4100e-01 3.3000e-01 2.3340e-05 5.2620e-05 6.2311e+01]
[3.7190e-12 3.4200e-01 3.3100e-01 2.2680e-05 5.0410e-05 6.2273e+01]
[3.7660e-12 3.4200e-01 3.3100e-01 2.2120e-05 4.9660e-05 6.2357e+01]
[4.1160e-12 3.4000e-01 3.2900e-01 2.4110e-05 5.2520e-05 6.2306e+01]
[3.7860e-12 3.4000e-01 3.3100e-01 2.2650e-05 5.0370e-05 6.2221e+01]
[3.8110e-12 3.4100e-01 3.3100e-01 2.2590e-05 5.1090e-05 6.2245e+01]
[4.075e-12 3.390e-01 3.290e-01 2.534e-05 5.402e-05 6.2

[2.1870e-12 3.4700e-01 3.4100e-01 8.4820e-06 2.5160e-05 6.1079e+01]
[2.1710e-12 3.4700e-01 3.4100e-01 8.7190e-06 2.6980e-05 6.1185e+01]
[2.0910e-12 3.4800e-01 3.4200e-01 8.2120e-06 2.1590e-05 6.1078e+01]
[2.0210e-12 3.4900e-01 3.4300e-01 8.4370e-06 2.2780e-05 6.1095e+01]
[2.1830e-12 3.4800e-01 3.4100e-01 8.8370e-06 2.5430e-05 6.1128e+01]
[2.1960e-12 3.4700e-01 3.4100e-01 8.3020e-06 2.7180e-05 6.1082e+01]
[2.1130e-12 3.4900e-01 3.4200e-01 8.2250e-06 2.1860e-05 6.1116e+01]
[2.1940e-12 3.4700e-01 3.4100e-01 8.8640e-06 2.5980e-05 6.1142e+01]
[2.1280e-12 3.4800e-01 3.4200e-01 8.7880e-06 2.4040e-05 6.1085e+01]
[2.0460e-12 3.4900e-01 3.4200e-01 8.3770e-06 2.3470e-05 6.1059e+01]
[2.1790e-12 3.4800e-01 3.4100e-01 8.3400e-06 2.4670e-05 6.1131e+01]
[2.1250e-12 3.4800e-01 3.4200e-01 8.5300e-06 2.4390e-05 6.1083e+01]
[2.1240e-12 3.4800e-01 3.4200e-01 8.4870e-06 2.4050e-05 6.1086e+01]
[2.0540e-12 3.4900e-01 3.4200e-01 8.6130e-06 2.6610e-05 6.1063e+01]
[2.043e-12 3.480e-01 3.430e-01 8.257e-06 2.345e-

### 2. WFV ( train + test )

In [81]:
# file_path == '2020_RDFWFV_20201029_V08.xlsx':
#     num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
#     num_output = 6
#     num_in_cycle_train = 50
#     num_in_cycle_test = 250
#     num_of_cycle_train = [50, 50, 100]
#     num_of_cycle_test = [2, 0, 2]
#     num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
#     num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
#     header = 0
  
# # WFV

# file_path == '2020_RDFWFV_20201029_V08.xlsx':
#     num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
#     num_output = 6
#     num_in_cycle_train = 50
#     num_in_cycle_test = 250
#     num_of_cycle_train = [50, 50, 100]
#     num_of_cycle_test = [2, 0, 2]
#     num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
#     num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
#     header = 0

file_path = '2020_RDFWFV_20201029_V08.xlsx'

#### TRAIN

## 1.read from excel
train_data_x = pd.read_excel('../'+file_path, sheet_name='WFV data', usecols="C:F", nrows=num_total_train[1]+1, header=header)
train_data_y = pd.read_excel('../'+file_path, sheet_name='WFV data', usecols="G:N", nrows=num_total_train[1]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_train , Y_all_train = np.zeros((num_total_train[1], num_input[1]-2)), np.zeros((num_total_train[1], num_output))
X_per_cycle_train, Y_per_cycle_train = np.zeros((num_of_cycle_train[1], num_input[1]-2)), np.zeros((num_of_cycle_train[1], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
train_data_y = train_data_y.drop('Idlo', axis=1)
train_data_y = train_data_y.drop('Idhi', axis=1)

print(train_data_y)

## 4. X_per_cycle
for i in range(num_of_cycle_train[1]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle_train[i] = train_data_x[i*num_in_cycle_train:i*num_in_cycle_train+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all_train = np.repeat(X_per_cycle_train,num_in_cycle_train,axis=0)


## 6. 인위적으로 one-hot 넣어주기
X_all_train = np.hstack((X_all_train, np.ones((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.ones((X_per_cycle_train.shape[0], 1))))

X_all_train = np.hstack((X_all_train, np.zeros((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.zeros((X_per_cycle_train.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle_train[1]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train] = train_data_y[i*num_in_cycle_train:(i+1)*num_in_cycle_train].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle_train[1]):
    Y_per_cycle_train[i] = np.mean(Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train],axis=0)
#     print(i, Y_per_cycle_train[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all_train.shape, "X per cycle data shape:", X_per_cycle_train.shape)
print("Y data shape: ", Y_all_train.shape, "Y per cycle data shape:", Y_per_cycle_train.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_train)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_train)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all_train)
data_train.append(Y_all_train)
data_train.append(X_per_cycle_train)
data_train.append(Y_per_cycle_train)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./wfv_train'+file_path, data_train)
print()

#### TEST

## 1.read from excel
test_data_x = pd.read_excel('../'+file_path, sheet_name='WFV testdata', usecols="B:E", nrows=num_total_test[1]+1, header=header)
test_data_y = pd.read_excel('../'+file_path, sheet_name='WFV testdata', usecols="F:M", nrows=num_total_test[1]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_test , Y_all_test = np.zeros((num_total_test[1], num_input[1]-2)), np.zeros((num_total_test[1], num_output))
X_per_cycle_test, Y_per_cycle_test = np.zeros((num_of_cycle_test[1], num_input[1]-2)), np.zeros((num_of_cycle_test[1], num_output))

## 3. DATA_X DATA_Y preprocessing
test_data_y = test_data_y.drop('Idlo', axis=1)
test_data_y = test_data_y.drop('Idhi', axis=1)

## 4. X_per_cycle, Y_all
for i in range(num_of_cycle_test[1]):
    X_per_cycle_test[i] = test_data_x[i*num_in_cycle_test:i*num_in_cycle_test+1].values
    
Y_all_test = test_data_y.values    

## 5. X_all, Y_per_cycle
for i in range(num_of_cycle_test[1]):
#     print(i*num_in_cycle_test, (i+1)*num_in_cycle_test)
#     print(X_per_cycle_test[i])
#     print(np.reshape(X_per_cycle_test[i], (1,3)).shape)
    X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test] = np.repeat(np.reshape(X_per_cycle_test[i], (1,3)), num_in_cycle_test, axis=0)
#     print(X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test])

    Y_per_cycle_test[i] = np.mean(Y_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test], axis=0)

## 6. 인위적으로 one-hot 넣어주기
X_all_test = np.hstack((X_all_test, np.ones((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.ones((X_per_cycle_test.shape[0], 1))))

X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))    
    
# for i in range(X_all_test.shape[0]):
#     print(X_all_test[i])
#     print(Y_all_test[i])
    
    
print("============ TEST Data load =============")
print()
print("X data shape: ", X_all_test.shape, "X per cycle data shape:", X_per_cycle_test.shape)
print("Y data shape: ", Y_all_test.shape, "Y per cycle data shape:", Y_per_cycle_test.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_test)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_test)))  
print()
print("============ TRAIN Data save =============")
print()
data_test = []
data_test.append(X_all_test)
data_test.append(Y_all_test)
data_test.append(X_per_cycle_test)
data_test.append(Y_per_cycle_test)
    
data_test = np.array(data_test)
print("saved data shape: ", data_test.shape)
np.save('./wfv_test'+file_path, data_test)
print()


# # RDF+WFV  
    

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     2.703000e-11  0.290  0.283  0.000032  0.000059  62.901
1     1.186000e-11  0.321  0.299  0.000032  0.000061  62.270
2     3.794000e-12  0.356  0.350  0.000030  0.000052  64.063
3     1.247000e-12  0.386  0.361  0.000028  0.000050  62.222
4     1.319000e-12  0.373  0.363  0.000029  0.000047  62.514
...            ...    ...    ...       ...       ...     ...
2496  3.999000e-12  0.337  0.333  0.000027  0.000051  62.634
2497  4.579000e-12  0.332  0.322  0.000028  0.000057  62.099
2498  3.646000e-12  0.339  0.331  0.000027  0.000053  62.261
2499  3.880000e-12  0.339  0.328  0.000027  0.000055  61.992
2500           NaN    NaN    NaN       NaN       NaN     NaN

[2501 rows x 6 columns]
============ TRAIN Data load =============

X data shape:  (2500, 6) X per cycle data shape: (50, 6)
Y data shape:  (2500, 6) Y per cycle data shape: (50, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============

sa

In [83]:
##### train
file_path = 'wfv_train2020_RDFWFV_20201029_V08.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

# test    
test_file_path = 'wfv_test2020_RDFWFV_20201029_V08.xlsx'

test_data = np.load(test_file_path+'.npy', allow_pickle=True)

test_X_all, test_Y_all, test_X_per_cycle, test_Y_per_cycle = test_data[0], test_data[1], test_data[2], test_data[3]

print(test_data[0].shape, test_data[1].shape, test_data[2].shape, test_data[3].shape)

for i in range(test_Y_all.shape[0]):
    print(test_Y_all[i])

(2500, 6) (2500, 6) (50, 6) (50, 6)
[2.7030e-11 2.9000e-01 2.8300e-01 3.2220e-05 5.8510e-05 6.2901e+01]
[1.186e-11 3.210e-01 2.990e-01 3.187e-05 6.053e-05 6.227e+01]
[3.7940e-12 3.5600e-01 3.5000e-01 3.0350e-05 5.2050e-05 6.4063e+01]
[1.2470e-12 3.8600e-01 3.6100e-01 2.8460e-05 4.9920e-05 6.2222e+01]
[1.3190e-12 3.7300e-01 3.6300e-01 2.8770e-05 4.7440e-05 6.2514e+01]
[2.9460e-12 3.4900e-01 3.4200e-01 3.0490e-05 5.1760e-05 6.2591e+01]
[1.654e-12 3.690e-01 3.590e-01 2.961e-05 4.989e-05 6.302e+01]
[1.0580e-10 2.5800e-01 2.4100e-01 3.5220e-05 7.4250e-05 6.2615e+01]
[2.4470e-11 2.9200e-01 2.8000e-01 3.4290e-05 6.5080e-05 6.2403e+01]
[2.8100e-11 2.9000e-01 2.8400e-01 3.3530e-05 6.2020e-05 6.3251e+01]
[3.4990e-13 4.0300e-01 3.9300e-01 2.8480e-05 4.5600e-05 6.2181e+01]
[7.3910e-12 3.2400e-01 3.2000e-01 3.0300e-05 5.5680e-05 6.3174e+01]
[3.1020e-11 2.9100e-01 2.7300e-01 3.2660e-05 6.3740e-05 6.2115e+01]
[2.1460e-11 3.2100e-01 2.8200e-01 3.1510e-05 6.2740e-05 6.2188e+01]
[5.338e-11 2.700e-01 2.6

[3.0250e-11 2.8300e-01 2.7700e-01 7.4020e-06 2.0070e-05 6.1496e+01]
[8.1880e-12 3.2500e-01 3.1900e-01 6.8370e-06 1.6900e-05 6.1572e+01]
[8.9060e-14 4.3300e-01 4.2600e-01 6.3320e-06 1.4320e-05 6.0944e+01]
[5.8670e-12 3.3300e-01 3.2900e-01 6.6370e-06 1.7060e-05 6.1599e+01]
[2.577e-11 3.070e-01 2.800e-01 7.081e-06 1.931e-05 6.141e+01]
[8.4000e-12 3.6900e-01 3.1200e-01 6.8720e-06 1.9500e-05 6.1519e+01]
[3.4650e-11 2.8700e-01 2.7700e-01 7.3840e-06 1.9570e-05 6.1667e+01]
[8.1720e-13 3.8700e-01 3.6800e-01 6.7700e-06 1.6310e-05 6.0608e+01]
[2.2680e-11 2.9700e-01 2.9500e-01 7.0700e-06 1.8060e-05 6.2348e+01]
[1.2870e-13 4.2000e-01 4.1500e-01 6.5770e-06 1.5660e-05 6.1022e+01]
[2.6330e-11 2.9700e-01 2.7800e-01 7.1380e-06 2.0030e-05 6.1023e+01]
[2.9510e-11 2.9400e-01 2.8400e-01 6.8800e-06 1.7290e-05 6.1322e+01]
[9.8490e-14 4.3000e-01 4.2000e-01 6.4320e-06 1.5680e-05 6.0764e+01]
[2.2630e-13 4.1600e-01 4.1500e-01 6.4420e-06 1.4580e-05 6.1974e+01]
[1.3090e-11 3.3600e-01 2.9800e-01 6.8800e-06 1.8820e-0

[3.0750e-12 3.4500e-01 3.3900e-01 1.2420e-05 3.1910e-05 6.1954e+01]
[2.7450e-12 3.4700e-01 3.4000e-01 1.2380e-05 3.1170e-05 6.1579e+01]
[3.5910e-12 3.3800e-01 3.2600e-01 1.2690e-05 3.4600e-05 6.1334e+01]
[1.9380e-12 3.5600e-01 3.4900e-01 1.2380e-05 3.0330e-05 6.1783e+01]
[2.3720e-12 3.5100e-01 3.4400e-01 1.2570e-05 3.3030e-05 6.1689e+01]
[1.7210e-12 3.6600e-01 3.5100e-01 1.2370e-05 3.0030e-05 6.1955e+01]
[5.4880e-12 3.3200e-01 3.2400e-01 1.2460e-05 3.0880e-05 6.2208e+01]
[1.2400e-12 3.6300e-01 3.5800e-01 1.2800e-05 3.2770e-05 6.1904e+01]
[2.3630e-12 3.5200e-01 3.4300e-01 1.2350e-05 3.0600e-05 6.1742e+01]
[5.7840e-12 3.2800e-01 3.1500e-01 1.2630e-05 3.2770e-05 6.1385e+01]
[2.7420e-12 3.4000e-01 3.3500e-01 1.2520e-05 3.2360e-05 6.1387e+01]
[5.0380e-12 3.3300e-01 3.2300e-01 1.2550e-05 3.2770e-05 6.1572e+01]
[1.6290e-12 3.6100e-01 3.5800e-01 1.2100e-05 2.9220e-05 6.2328e+01]
[3.4730e-12 3.3900e-01 3.3600e-01 1.2700e-05 3.1400e-05 6.2546e+01]
[3.4600e-12 3.4000e-01 3.3200e-01 1.2430e-05 3.1

### 3. RDF+WFV ( train + test )

In [85]:
# file_path == '2020_RDFWFV_20201029_V08.xlsx':
#     num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
#     num_output = 6
#     num_in_cycle_train = 50
#     num_in_cycle_test = 250
#     num_of_cycle_train = [50, 50, 100]
#     num_of_cycle_test = [2, 0, 2]
#     num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
#     num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
#     header = 0

file_path = '2020_RDFWFV_20201029_V08.xlsx'

#### TRAIN

## 1.read from excel
train_data_x = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="C:F", nrows=num_total_train[2]+1, header=header)
train_data_y = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="G:N", nrows=num_total_train[2]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_train , Y_all_train = np.zeros((num_total_train[2], num_input[2]-2)), np.zeros((num_total_train[2], num_output))
X_per_cycle_train, Y_per_cycle_train = np.zeros((num_of_cycle_train[2], num_input[2]-2)), np.zeros((num_of_cycle_train[2], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
train_data_y = train_data_y.drop('Idlo', axis=1)
train_data_y = train_data_y.drop('Idhi', axis=1)

print(train_data_y)

## 4. X_per_cycle
for i in range(num_of_cycle_train[2]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle_train[i] = train_data_x[i*num_in_cycle_train:i*num_in_cycle_train+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all_train = np.repeat(X_per_cycle_train,num_in_cycle_train,axis=0)


## 6. 인위적으로 one-hot 넣어주기
X_all_train = np.hstack((X_all_train, np.ones((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.ones((X_per_cycle_train.shape[0], 1))))

X_all_train = np.hstack((X_all_train, np.zeros((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.zeros((X_per_cycle_train.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle_train[2]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train] = train_data_y[i*num_in_cycle_train:(i+1)*num_in_cycle_train].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle_train[2]):
    Y_per_cycle_train[i] = np.mean(Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train],axis=0)
#     print(i, Y_per_cycle_train[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all_train.shape, "X per cycle data shape:", X_per_cycle_train.shape)
print("Y data shape: ", Y_all_train.shape, "Y per cycle data shape:", Y_per_cycle_train.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_train)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_train)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all_train)
data_train.append(Y_all_train)
data_train.append(X_per_cycle_train)
data_train.append(Y_per_cycle_train)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdfwfv_train'+file_path, data_train)
print()

#### TEST

## 1.read from excel
test_data_x = pd.read_excel('../'+file_path, sheet_name='RDFWFV testdata', usecols="B:E", nrows=num_total_test[2]+1, header=header)
test_data_y = pd.read_excel('../'+file_path, sheet_name='RDFWFV testdata', usecols="F:M", nrows=num_total_test[2]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_test , Y_all_test = np.zeros((num_total_test[2], num_input[2]-2)), np.zeros((num_total_test[2], num_output))
X_per_cycle_test, Y_per_cycle_test = np.zeros((num_of_cycle_test[2], num_input[2]-2)), np.zeros((num_of_cycle_test[2], num_output))

## 3. DATA_X DATA_Y preprocessing
test_data_y = test_data_y.drop('Idlo', axis=1)
test_data_y = test_data_y.drop('Idhi', axis=1)

## 4. X_per_cycle, Y_all
for i in range(num_of_cycle_test[2]):
    X_per_cycle_test[i] = test_data_x[i*num_in_cycle_test:i*num_in_cycle_test+1].values
    
Y_all_test = test_data_y.values    

## 5. X_all, Y_per_cycle
for i in range(num_of_cycle_test[2]):
#     print(i*num_in_cycle_test, (i+1)*num_in_cycle_test)
#     print(X_per_cycle_test[i])
#     print(np.reshape(X_per_cycle_test[i], (1,3)).shape)
    X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test] = np.repeat(np.reshape(X_per_cycle_test[i], (1,4)), num_in_cycle_test, axis=0)
#     print(X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test])

    Y_per_cycle_test[i] = np.mean(Y_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test], axis=0)

## 6. 인위적으로 one-hot 넣어주기
X_all_test = np.hstack((X_all_test, np.ones((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.ones((X_per_cycle_test.shape[0], 1))))

X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))    
    
# for i in range(X_all_test.shape[0]):
#     print(X_all_test[i])
#     print(Y_all_test[i])
    
    
print("============ TEST Data load =============")
print()
print("X data shape: ", X_all_test.shape, "X per cycle data shape:", X_per_cycle_test.shape)
print("Y data shape: ", Y_all_test.shape, "Y per cycle data shape:", Y_per_cycle_test.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_test)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_test)))  
print()
print("============ TRAIN Data save =============")
print()
data_test = []
data_test.append(X_all_test)
data_test.append(Y_all_test)
data_test.append(X_per_cycle_test)
data_test.append(Y_per_cycle_test)
    
data_test = np.array(data_test)
print("saved data shape: ", data_test.shape)
np.save('./rdfwfv_test'+file_path, data_test)
print()


# # RDF+WFV  
    

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     3.570000e-11  0.285  0.277  0.000024  0.000057  63.283
1     1.651000e-11  0.316  0.293  0.000024  0.000056  62.388
2     1.117000e-11  0.338  0.325  0.000023  0.000051  65.969
3     1.916000e-12  0.379  0.354  0.000022  0.000046  63.341
4     2.597000e-12  0.357  0.347  0.000023  0.000049  63.126
...            ...    ...    ...       ...       ...     ...
4995  3.492000e-12  0.340  0.326  0.000009  0.000027  60.761
4996  2.046000e-12  0.355  0.345  0.000008  0.000022  61.140
4997  2.312000e-12  0.356  0.347  0.000008  0.000024  61.816
4998  1.211000e-11  0.310  0.302  0.000010  0.000030  61.313
4999  3.424000e-12  0.360  0.349  0.000009  0.000026  63.035

[5000 rows x 6 columns]
============ TRAIN Data load =============

X data shape:  (5000, 6) X per cycle data shape: (100, 6)
Y data shape:  (5000, 6) Y per cycle data shape: (100, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============



In [86]:
##### train
file_path = 'rdfwfv_train2020_RDFWFV_20201029_V08.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

# test    
test_file_path = 'rdfwfv_test2020_RDFWFV_20201029_V08.xlsx'

test_data = np.load(test_file_path+'.npy', allow_pickle=True)

test_X_all, test_Y_all, test_X_per_cycle, test_Y_per_cycle = test_data[0], test_data[1], test_data[2], test_data[3]

print(test_data[0].shape, test_data[1].shape, test_data[2].shape, test_data[3].shape)

for i in range(test_Y_all.shape[0]):
    print(test_Y_all[i])

(5000, 6) (5000, 6) (100, 6) (100, 6)
[3.5700e-11 2.8500e-01 2.7700e-01 2.4430e-05 5.7230e-05 6.3283e+01]
[1.6510e-11 3.1600e-01 2.9300e-01 2.3700e-05 5.6060e-05 6.2388e+01]
[1.1170e-11 3.3800e-01 3.2500e-01 2.3180e-05 5.1100e-05 6.5969e+01]
[1.9160e-12 3.7900e-01 3.5400e-01 2.1630e-05 4.6370e-05 6.3341e+01]
[2.5970e-12 3.5700e-01 3.4700e-01 2.3220e-05 4.8530e-05 6.3126e+01]
[5.1690e-12 3.4000e-01 3.3200e-01 2.3740e-05 5.2030e-05 6.4033e+01]
[2.1280e-12 3.6500e-01 3.5400e-01 2.3860e-05 4.9420e-05 6.3262e+01]
[1.9850e-10 2.4200e-01 2.2800e-01 2.7770e-05 7.3630e-05 6.3776e+01]
[5.3330e-11 2.7700e-01 2.6500e-01 2.5750e-05 6.0930e-05 6.3092e+01]
[3.9620e-11 2.8400e-01 2.7700e-01 2.5330e-05 5.9300e-05 6.3403e+01]
[3.4120e-13 4.0900e-01 3.9600e-01 2.0560e-05 4.0210e-05 6.2841e+01]
[1.1660e-11 3.1700e-01 3.1100e-01 2.3270e-05 5.3120e-05 6.3704e+01]
[4.5650e-11 2.7700e-01 2.6300e-01 2.6480e-05 6.5540e-05 6.2148e+01]
[3.0710e-11 3.0000e-01 2.7400e-01 2.4430e-05 5.9970e-05 6.2568e+01]
[7.9300e-1

[2.250e-12 3.530e-01 3.480e-01 8.964e-06 2.522e-05 6.178e+01]
[3.5100e-12 3.3800e-01 3.3400e-01 8.5770e-06 2.4250e-05 6.1711e+01]
[3.3900e-12 3.4000e-01 3.3200e-01 8.5000e-06 2.5490e-05 6.1341e+01]
[7.4490e-12 3.2800e-01 3.1500e-01 8.4610e-06 2.2570e-05 6.1369e+01]
[5.6350e-12 3.2500e-01 3.1700e-01 8.8080e-06 2.6830e-05 6.0788e+01]
[1.5130e-12 3.6400e-01 3.5000e-01 8.5290e-06 2.4650e-05 6.1433e+01]
[3.4800e-12 3.3800e-01 3.2900e-01 8.1520e-06 2.4620e-05 6.1028e+01]
[1.6870e-12 3.5700e-01 3.4600e-01 8.4270e-06 2.3750e-05 6.0858e+01]
[6.9690e-12 3.2100e-01 3.1400e-01 8.8670e-06 2.6790e-05 6.1267e+01]
[4.4870e-13 3.8800e-01 3.8100e-01 7.9020e-06 2.1890e-05 6.1056e+01]
[2.1000e-12 3.5600e-01 3.4000e-01 8.2760e-06 2.4090e-05 6.1021e+01]
[3.3970e-12 3.4100e-01 3.2700e-01 8.5950e-06 2.7730e-05 6.0906e+01]
[2.5170e-12 3.4700e-01 3.3800e-01 9.0760e-06 2.7230e-05 6.1054e+01]
[1.1730e-12 3.6700e-01 3.5900e-01 8.2200e-06 2.2710e-05 6.1522e+01]
[4.1440e-12 3.3400e-01 3.2700e-01 8.3250e-06 2.5210e-0

[6.1570e-11 2.6700e-01 2.5600e-01 2.1480e-05 5.9670e-05 6.2399e+01]
[1.6530e-12 3.6700e-01 3.5100e-01 1.8720e-05 4.3620e-05 6.1919e+01]
[2.2400e-11 3.0600e-01 2.9300e-01 1.9790e-05 4.7440e-05 6.2913e+01]
[3.0780e-13 4.0600e-01 3.9800e-01 1.7640e-05 3.7480e-05 6.2285e+01]
[1.6640e-11 3.0400e-01 2.9400e-01 1.9120e-05 4.8610e-05 6.2134e+01]
[6.6080e-11 2.7000e-01 2.5800e-01 2.0670e-05 5.4080e-05 6.2635e+01]
[2.6690e-13 4.1200e-01 4.0000e-01 1.6880e-05 3.5650e-05 6.2175e+01]
[1.0420e-12 3.7600e-01 3.7000e-01 1.7820e-05 4.0690e-05 6.2779e+01]
[2.2200e-11 3.1800e-01 2.8400e-01 1.9600e-05 4.9470e-05 6.2077e+01]
[1.4410e-12 3.8100e-01 3.5600e-01 1.7800e-05 4.0140e-05 6.1641e+01]
[2.8140e-12 3.5600e-01 3.3800e-01 2.1680e-05 4.7440e-05 6.2459e+01]
[7.235e-12 3.360e-01 3.110e-01 1.915e-05 4.823e-05 6.223e+01]
[1.4550e-11 3.1100e-01 3.0600e-01 2.0220e-05 4.7750e-05 6.3147e+01]
[7.5960e-13 3.9200e-01 3.7500e-01 1.8820e-05 4.1700e-05 6.3148e+01]
[1.1580e-11 3.2000e-01 3.0800e-01 2.2010e-05 4.7390e-0

[2.4360e-11 3.1300e-01 2.8100e-01 2.2160e-05 5.4400e-05 6.2246e+01]
[6.7280e-11 2.6500e-01 2.5400e-01 2.5690e-05 6.4210e-05 6.2501e+01]
[2.2960e-12 3.6000e-01 3.4600e-01 2.3430e-05 4.7660e-05 6.2195e+01]
[2.5480e-11 3.0600e-01 2.8900e-01 2.1880e-05 5.0530e-05 6.3091e+01]
[3.5610e-13 4.0500e-01 3.9700e-01 2.0770e-05 4.0390e-05 6.2545e+01]
[1.563e-11 3.090e-01 2.990e-01 2.190e-05 5.127e-05 6.292e+01]
[7.3730e-11 2.6800e-01 2.5700e-01 2.4650e-05 5.9640e-05 6.2985e+01]
[3.523e-13 4.100e-01 3.950e-01 2.122e-05 4.062e-05 6.222e+01]
[1.1720e-12 3.7300e-01 3.6600e-01 2.1620e-05 4.4950e-05 6.2571e+01]
[2.8600e-11 3.0500e-01 2.7800e-01 2.3320e-05 5.3860e-05 6.2278e+01]
[1.6110e-12 3.8000e-01 3.5400e-01 1.9980e-05 4.2790e-05 6.1902e+01]
[6.4150e-12 3.3500e-01 3.2000e-01 2.3860e-05 5.3450e-05 6.3062e+01]
[8.3860e-12 3.3300e-01 3.0800e-01 2.1820e-05 5.2390e-05 6.2369e+01]
[1.420e-11 3.120e-01 3.060e-01 2.147e-05 4.883e-05 6.339e+01]
[3.1430e-12 3.4800e-01 3.3700e-01 2.3440e-05 5.0160e-05 6.2182e+01

[8.5440e-12 3.1500e-01 3.0700e-01 8.6890e-06 2.3840e-05 6.1028e+01]
[8.5120e-12 3.2300e-01 3.1800e-01 8.6770e-06 2.4510e-05 6.2038e+01]
[3.2410e-12 3.4100e-01 3.3800e-01 9.2110e-06 2.6870e-05 6.1841e+01]
[6.848e-12 3.240e-01 3.160e-01 8.811e-06 2.454e-05 6.167e+01]
[1.6800e-12 3.5400e-01 3.4800e-01 8.5110e-06 2.3530e-05 6.0756e+01]
[5.6730e-12 3.3600e-01 3.1500e-01 8.6740e-06 2.6950e-05 6.0971e+01]
[1.9510e-11 2.9300e-01 2.8500e-01 8.7560e-06 2.7970e-05 6.1296e+01]
[5.6540e-13 3.8200e-01 3.7600e-01 8.4800e-06 2.2760e-05 6.1167e+01]
[9.4060e-13 3.7700e-01 3.7600e-01 8.3850e-06 2.1060e-05 6.2289e+01]
[4.9910e-13 3.9400e-01 3.8900e-01 8.8190e-06 2.3660e-05 6.2258e+01]
[2.7890e-12 3.4500e-01 3.3400e-01 8.6780e-06 2.5300e-05 6.1002e+01]
[1.6090e-11 3.0800e-01 2.9500e-01 8.8380e-06 2.5540e-05 6.1709e+01]
[1.1410e-12 3.7100e-01 3.5900e-01 8.6600e-06 2.4910e-05 6.1278e+01]
[5.5190e-13 3.8800e-01 3.8100e-01 8.7910e-06 2.4610e-05 6.2279e+01]
[3.7790e-12 3.5100e-01 3.3300e-01 8.5120e-06 2.5360e-0

[8.3370e-12 3.1400e-01 3.0700e-01 5.9860e-06 1.6290e-05 6.0648e+01]
[1.1890e-12 3.7900e-01 3.6400e-01 5.2960e-06 1.2390e-05 6.0944e+01]
[3.8770e-12 3.5500e-01 3.3300e-01 5.3130e-06 1.4070e-05 6.0694e+01]
[6.954e-12 3.340e-01 3.300e-01 5.685e-06 1.447e-05 6.183e+01]
[1.2410e-13 4.2500e-01 4.1900e-01 5.2360e-06 1.2320e-05 6.1023e+01]
[4.1140e-12 3.4200e-01 3.3100e-01 5.4530e-06 1.5420e-05 6.1193e+01]
[5.6620e-13 3.9200e-01 3.8900e-01 5.4230e-06 1.2530e-05 6.2402e+01]
[7.8210e-13 3.7900e-01 3.7700e-01 5.8230e-06 1.3720e-05 6.1749e+01]
[8.6850e-13 3.7400e-01 3.7100e-01 5.4120e-06 1.2730e-05 6.1418e+01]
[6.6260e-13 3.8600e-01 3.7400e-01 5.7100e-06 1.4290e-05 6.0927e+01]
[3.7570e-12 3.4800e-01 3.3200e-01 6.0630e-06 1.6540e-05 6.1144e+01]
[1.9760e-11 2.9900e-01 2.8700e-01 5.8830e-06 1.7140e-05 6.1203e+01]
[8.2770e-13 3.7100e-01 3.6600e-01 5.8730e-06 1.4980e-05 6.0567e+01]
[1.6740e-11 2.9900e-01 2.9000e-01 6.5910e-06 1.8350e-05 6.0692e+01]
[1.1280e-12 3.7200e-01 3.6900e-01 6.1270e-06 1.4510e-0

In [9]:
# file_path == '2020_RDFWFV_20201029_V08.xlsx':
#     num_input = [5, 6, 6] # RDF, WFV, RDF+WFV
#     num_output = 6
#     num_in_cycle_train = 50
#     num_in_cycle_test = 250
#     num_of_cycle_train = [50, 50, 100]
#     num_of_cycle_test = [2, 0, 2]
#     num_total_train = [x*num_in_cycle_train for x in num_of_cycle_train]
#     num_total_test = [x*num_in_cycle_test for x in num_of_cycle_test]
#     header = 0

file_path = '2020_RDFWFV_20201101_V09.xlsx'

#### TRAIN

## 1.read from excel
train_data_x = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="C:F", nrows=num_total_train[2]+1, header=header)
train_data_y = pd.read_excel('../'+file_path, sheet_name='RDFWFV data', usecols="G:N", nrows=num_total_train[2]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_train , Y_all_train = np.zeros((num_total_train[2], num_input[2]-2)), np.zeros((num_total_train[2], num_output))
X_per_cycle_train, Y_per_cycle_train = np.zeros((num_of_cycle_train[2], num_input[2]-2)), np.zeros((num_of_cycle_train[2], num_output))

## 3. DATA_X DATA_Y preprocessing
# Y: IDLO, IDHI
train_data_y = train_data_y.drop('Idlo', axis=1)
train_data_y = train_data_y.drop('Idhi', axis=1)

print(train_data_y)

## 4. X_per_cycle
for i in range(num_of_cycle_train[2]):
#     print(i*num_in_cycle_train, i*num_in_cycle_train+1)
    X_per_cycle_train[i] = train_data_x[i*num_in_cycle_train:i*num_in_cycle_train+1].values
#     print(X_per_cycle_train[i])

## 5. X_all
X_all_train = np.repeat(X_per_cycle_train,num_in_cycle_train,axis=0)


## 6. 인위적으로 one-hot 넣어주기
X_all_train = np.hstack((X_all_train, np.ones((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.ones((X_per_cycle_train.shape[0], 1))))

X_all_train = np.hstack((X_all_train, np.zeros((X_all_train.shape[0], 1))))
X_per_cycle_train = np.hstack((X_per_cycle_train, np.zeros((X_per_cycle_train.shape[0], 1))))


## 7. Y_all
for i in range(num_of_cycle_train[2]):
#     print(i*num_in_cycle_train,(i+1)*num_in_cycle_train)
    Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train] = train_data_y[i*num_in_cycle_train:(i+1)*num_in_cycle_train].values
#     print(Y_all_train[i*num_in_cycle_train])

## 8. Y_per_cycle    
for i in range(num_of_cycle_train[2]):
    Y_per_cycle_train[i] = np.mean(Y_all_train[i*num_in_cycle_train:(i+1)*num_in_cycle_train],axis=0)
#     print(i, Y_per_cycle_train[i])

print("============ TRAIN Data load =============")
print()
print("X data shape: ", X_all_train.shape, "X per cycle data shape:", X_per_cycle_train.shape)
print("Y data shape: ", Y_all_train.shape, "Y per cycle data shape:", Y_per_cycle_train.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_train)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_train)))  
print()
print("============ TRAIN Data save =============")
print()
data_train = []
data_train.append(X_all_train)
data_train.append(Y_all_train)
data_train.append(X_per_cycle_train)
data_train.append(Y_per_cycle_train)
    
data_train = np.array(data_train)
print("saved data shape: ", data_train.shape)
np.save('./rdfwfv_train'+file_path, data_train)
print()

#### TEST

## 1.read from excel
test_data_x = pd.read_excel('../'+file_path, sheet_name='RDFWFV testdata', usecols="B:E", nrows=num_total_test[2]+1, header=header)
test_data_y = pd.read_excel('../'+file_path, sheet_name='RDFWFV testdata', usecols="F:M", nrows=num_total_test[2]+1, header=header)

## 2.one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all_test , Y_all_test = np.zeros((num_total_test[2], num_input[2]-2)), np.zeros((num_total_test[2], num_output))
X_per_cycle_test, Y_per_cycle_test = np.zeros((num_of_cycle_test[2], num_input[2]-2)), np.zeros((num_of_cycle_test[2], num_output))

## 3. DATA_X DATA_Y preprocessing
test_data_y = test_data_y.drop('Idlo', axis=1)
test_data_y = test_data_y.drop('Idhi', axis=1)

## 4. X_per_cycle, Y_all
for i in range(num_of_cycle_test[2]):
    X_per_cycle_test[i] = test_data_x[i*num_in_cycle_test:i*num_in_cycle_test+1].values
    
Y_all_test = test_data_y.values    

## 5. X_all, Y_per_cycle
for i in range(num_of_cycle_test[2]):
#     print(i*num_in_cycle_test, (i+1)*num_in_cycle_test)
#     print(X_per_cycle_test[i])
#     print(np.reshape(X_per_cycle_test[i], (1,3)).shape)
    X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test] = np.repeat(np.reshape(X_per_cycle_test[i], (1,4)), num_in_cycle_test, axis=0)
#     print(X_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test])

    Y_per_cycle_test[i] = np.mean(Y_all_test[i*num_in_cycle_test:(i+1)*num_in_cycle_test], axis=0)

## 6. 인위적으로 one-hot 넣어주기
X_all_test = np.hstack((X_all_test, np.ones((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.ones((X_per_cycle_test.shape[0], 1))))

X_all_test = np.hstack((X_all_test, np.zeros((X_all_test.shape[0], 1))))
X_per_cycle_test = np.hstack((X_per_cycle_test, np.zeros((X_per_cycle_test.shape[0], 1))))    
    
# for i in range(X_all_test.shape[0]):
#     print(X_all_test[i])
#     print(Y_all_test[i])
    
    
print("============ TEST Data load =============")
print()
print("X data shape: ", X_all_test.shape, "X per cycle data shape:", X_per_cycle_test.shape)
print("Y data shape: ", Y_all_test.shape, "Y per cycle data shape:", Y_per_cycle_test.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all_test)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all_test)))  
print()
print("============ TRAIN Data save =============")
print()
data_test = []
data_test.append(X_all_test)
data_test.append(Y_all_test)
data_test.append(X_per_cycle_test)
data_test.append(Y_per_cycle_test)
    
data_test = np.array(data_test)
print("saved data shape: ", data_test.shape)
np.save('./rdfwfv_test'+file_path, data_test)
print()


# # RDF+WFV  
    

              Ioff  Vtlin  Vtsat     Idlin     Idsat      SS
0     3.570000e-11  0.285  0.277  0.000024  0.000057  63.283
1     1.651000e-11  0.316  0.293  0.000024  0.000056  62.388
2     1.117000e-11  0.338  0.325  0.000023  0.000051  65.969
3     1.916000e-12  0.379  0.354  0.000022  0.000046  63.341
4     2.597000e-12  0.357  0.347  0.000023  0.000049  63.126
...            ...    ...    ...       ...       ...     ...
4995  3.492000e-12  0.340  0.326  0.000009  0.000027  60.761
4996  2.046000e-12  0.355  0.345  0.000008  0.000022  61.140
4997  2.312000e-12  0.356  0.347  0.000008  0.000024  61.816
4998  1.211000e-11  0.310  0.302  0.000010  0.000030  61.313
4999  3.424000e-12  0.360  0.349  0.000009  0.000026  63.035

[5000 rows x 6 columns]
============ TRAIN Data load =============

X data shape:  (5000, 6) X per cycle data shape: (100, 6)
Y data shape:  (5000, 6) Y per cycle data shape: (100, 6)
any nan in X?:  []
any nan in Y?:  []

============ TRAIN Data save =============



In [15]:
##### train
file_path = 'rdfwfv_train2020_RDFWFV_20201101_V09.xlsx'

data = np.load(file_path+'.npy', allow_pickle=True)

X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

# test    
test_file_path = 'rdfwfv_test2020_RDFWFV_20201101_V09.xlsx'

test_data = np.load(test_file_path+'.npy', allow_pickle=True)

test_X_all, test_Y_all, test_X_per_cycle, test_Y_per_cycle = test_data[0], test_data[1], test_data[2], test_data[3]

print(test_data[0].shape, test_data[1].shape, test_data[2].shape, test_data[3].shape)

for i in range(test_Y_all.shape[0]):
    print(test_Y_all[i])

(5000, 6) (5000, 6) (100, 6) (100, 6)
[3.5700e-11 2.8500e-01 2.7700e-01 2.4430e-05 5.7230e-05 6.3283e+01]
[1.6510e-11 3.1600e-01 2.9300e-01 2.3700e-05 5.6060e-05 6.2388e+01]
[1.1170e-11 3.3800e-01 3.2500e-01 2.3180e-05 5.1100e-05 6.5969e+01]
[1.9160e-12 3.7900e-01 3.5400e-01 2.1630e-05 4.6370e-05 6.3341e+01]
[2.5970e-12 3.5700e-01 3.4700e-01 2.3220e-05 4.8530e-05 6.3126e+01]
[5.1690e-12 3.4000e-01 3.3200e-01 2.3740e-05 5.2030e-05 6.4033e+01]
[2.1280e-12 3.6500e-01 3.5400e-01 2.3860e-05 4.9420e-05 6.3262e+01]
[1.9850e-10 2.4200e-01 2.2800e-01 2.7770e-05 7.3630e-05 6.3776e+01]
[5.3330e-11 2.7700e-01 2.6500e-01 2.5750e-05 6.0930e-05 6.3092e+01]
[3.9620e-11 2.8400e-01 2.7700e-01 2.5330e-05 5.9300e-05 6.3403e+01]
[3.4120e-13 4.0900e-01 3.9600e-01 2.0560e-05 4.0210e-05 6.2841e+01]
[1.1660e-11 3.1700e-01 3.1100e-01 2.3270e-05 5.3120e-05 6.3704e+01]
[4.5650e-11 2.7700e-01 2.6300e-01 2.6480e-05 6.5540e-05 6.2148e+01]
[3.0710e-11 3.0000e-01 2.7400e-01 2.4430e-05 5.9970e-05 6.2568e+01]
[7.9300e-1

[5.5320e-12 3.3100e-01 3.2000e-01 2.5120e-05 5.5490e-05 6.2475e+01]
[3.8730e-12 3.3900e-01 3.3100e-01 2.4430e-05 5.4030e-05 6.2509e+01]
[3.1540e-12 3.4500e-01 3.3600e-01 2.5390e-05 5.3790e-05 6.2385e+01]
[3.5220e-12 3.4400e-01 3.3300e-01 2.4490e-05 5.3100e-05 6.2352e+01]
[4.2080e-12 3.3800e-01 3.3000e-01 2.4850e-05 5.3970e-05 6.2675e+01]
[5.4560e-12 3.3300e-01 3.2300e-01 2.6270e-05 5.7520e-05 6.2606e+01]
[4.9170e-12 3.3600e-01 3.2600e-01 2.6470e-05 5.6850e-05 6.2634e+01]
[6.4310e-12 3.2800e-01 3.2000e-01 2.6440e-05 5.7380e-05 6.2895e+01]
[5.4060e-12 3.3400e-01 3.2200e-01 2.4500e-05 5.6000e-05 6.2542e+01]
[3.3730e-12 3.4400e-01 3.3600e-01 2.6180e-05 5.5620e-05 6.2701e+01]
[3.8450e-12 3.4200e-01 3.3300e-01 2.9720e-05 5.9290e-05 6.2715e+01]
[4.2940e-12 3.3800e-01 3.2700e-01 2.3740e-05 5.3440e-05 6.2401e+01]
[4.5050e-12 3.3700e-01 3.2900e-01 2.4540e-05 5.4760e-05 6.2731e+01]
[4.6420e-12 3.3700e-01 3.2600e-01 2.7040e-05 5.8070e-05 6.2482e+01]
[3.9030e-12 3.4100e-01 3.3200e-01 2.5100e-05 5.5

[3.6160e-11 2.8500e-01 2.7700e-01 9.4030e-06 2.4820e-05 6.1792e+01]
[1.5840e-13 4.1800e-01 4.1000e-01 9.0720e-06 2.3590e-05 6.1301e+01]
[6.1390e-13 3.8600e-01 3.8200e-01 8.9430e-06 2.3730e-05 6.1766e+01]
[1.2360e-11 3.2600e-01 3.0000e-01 9.2460e-06 2.7280e-05 6.1284e+01]
[1.8860e-12 3.8100e-01 3.5100e-01 8.7270e-06 2.5190e-05 6.1226e+01]
[1.0500e-12 3.7300e-01 3.6100e-01 9.3830e-06 2.7380e-05 6.1157e+01]
[4.5860e-12 3.4200e-01 3.2100e-01 9.9790e-06 2.9240e-05 6.1343e+01]
[8.6810e-12 3.1800e-01 3.1400e-01 9.8800e-06 3.0610e-05 6.2045e+01]
[3.3040e-13 4.0200e-01 3.9000e-01 9.5340e-06 2.6290e-05 6.1378e+01]
[6.766e-12 3.270e-01 3.190e-01 9.564e-06 2.763e-05 6.141e+01]
[1.497e-11 3.090e-01 3.040e-01 1.015e-05 2.927e-05 6.208e+01]
[4.9580e-11 2.7400e-01 2.6200e-01 9.4850e-06 3.0360e-05 6.1537e+01]
[7.456e-12 3.370e-01 3.320e-01 9.325e-06 2.643e-05 6.298e+01]
[2.9120e-12 3.4900e-01 3.4000e-01 9.5480e-06 2.7790e-05 6.1818e+01]
[1.6270e-11 3.0600e-01 2.8800e-01 1.0690e-05 3.4500e-05 6.1508e+01

[1.5580e-12 3.6400e-01 3.6200e-01 8.8830e-06 2.4840e-05 6.2395e+01]
[5.3010e-12 3.3700e-01 3.1600e-01 8.6550e-06 2.6530e-05 6.1006e+01]
[6.7940e-12 3.2600e-01 3.1700e-01 8.6460e-06 2.3830e-05 6.1585e+01]
[2.2250e-12 3.7000e-01 3.6100e-01 8.5400e-06 2.3370e-05 6.2373e+01]
[3.6170e-12 3.5600e-01 3.3200e-01 9.3240e-06 2.8720e-05 6.1889e+01]
[1.4220e-12 3.6500e-01 3.5700e-01 8.0110e-06 2.0290e-05 6.1489e+01]
[2.4260e-11 2.9600e-01 2.8800e-01 2.4000e-05 5.4630e-05 6.2992e+01]
[3.0980e-11 2.9900e-01 2.7600e-01 2.5790e-05 5.8890e-05 6.2535e+01]
[1.3510e-11 3.1700e-01 3.0100e-01 2.4280e-05 5.5320e-05 6.3025e+01]
[1.6760e-12 3.7900e-01 3.5900e-01 2.0950e-05 4.4110e-05 6.3069e+01]
[4.7640e-12 3.4300e-01 3.3100e-01 2.2940e-05 4.9890e-05 6.2884e+01]
[4.3080e-12 3.4000e-01 3.3400e-01 2.2080e-05 4.8150e-05 6.2849e+01]
[2.1050e-12 3.6500e-01 3.5400e-01 2.2270e-05 4.6320e-05 6.3107e+01]
[1.4480e-10 2.4400e-01 2.3400e-01 2.5600e-05 6.6600e-05 6.2729e+01]
[6.0870e-11 2.7300e-01 2.6200e-01 2.4840e-05 5.7

[1.7430e-12 3.6400e-01 3.5000e-01 5.3990e-06 1.3480e-05 6.0947e+01]
[2.5040e-12 3.5000e-01 3.4300e-01 5.5630e-06 1.3310e-05 6.1143e+01]
[2.5570e-12 3.4200e-01 3.3600e-01 5.8740e-06 1.6060e-05 6.0761e+01]
[2.0510e-12 3.5400e-01 3.4800e-01 6.0840e-06 1.6190e-05 6.1262e+01]
[2.6290e-12 3.4600e-01 3.3700e-01 5.5970e-06 1.2990e-05 6.0726e+01]
[1.6150e-12 3.6200e-01 3.5400e-01 5.9130e-06 1.6280e-05 6.1013e+01]
[1.6650e-12 3.6000e-01 3.4800e-01 5.8870e-06 1.5360e-05 6.0926e+01]
[2.2830e-12 3.4800e-01 3.4300e-01 5.8560e-06 1.7430e-05 6.1132e+01]
[3.0420e-12 3.4500e-01 3.3500e-01 5.7410e-06 1.5400e-05 6.0967e+01]
[1.0140e-12 3.7100e-01 3.6500e-01 5.7870e-06 1.4790e-05 6.1036e+01]
[2.2160e-12 3.5200e-01 3.4600e-01 5.6380e-06 1.4640e-05 6.1263e+01]
[1.575e-12 3.540e-01 3.510e-01 6.350e-06 1.632e-05 6.085e+01]
[3.966e-12 3.370e-01 3.270e-01 6.160e-06 1.670e-05 6.099e+01]
[2.7250e-12 3.5000e-01 3.4400e-01 5.8100e-06 1.4820e-05 6.1214e+01]
[2.7480e-12 3.5800e-01 3.4400e-01 5.7280e-06 1.5270e-05 6.11

...

[2.0230e-12 3.5800e-01 3.4600e-01 1.8630e-05 4.4090e-05 6.2139e+01]
[5.9330e-13 3.8800e-01 3.7800e-01 1.7960e-05 3.9800e-05 6.2132e+01]
[6.2490e-12 3.4000e-01 3.2400e-01 1.7590e-05 4.2600e-05 6.2403e+01]
[2.8300e-12 3.5300e-01 3.4400e-01 1.9680e-05 4.3730e-05 6.2553e+01]
[7.3120e-12 3.2600e-01 3.1100e-01 2.0710e-05 5.0070e-05 6.2281e+01]
[3.5620e-12 3.4300e-01 3.3500e-01 1.8710e-05 4.3350e-05 6.2362e+01]
[2.666e-11 3.010e-01 2.870e-01 1.814e-05 4.516e-05 6.249e+01]
[2.0250e-12 3.6800e-01 3.5400e-01 1.7440e-05 4.1330e-05 6.3671e+01]
[2.4420e-11 3.0700e-01 2.8500e-01 1.9940e-05 4.9830e-05 6.2281e+01]
[2.5390e-12 3.5000e-01 3.4400e-01 2.0020e-05 4.6560e-05 6.2551e+01]
[5.6910e-12 3.3000e-01 3.2400e-01 1.9030e-05 4.8120e-05 6.2519e+01]
[7.6380e-12 3.2200e-01 3.1300e-01 1.9370e-05 4.8090e-05 6.2423e+01]
[8.1530e-12 3.2500e-01 3.1000e-01 1.9480e-05 4.9530e-05 6.2015e+01]
[5.9090e-12 3.3600e-01 3.2100e-01 1.8050e-05 4.4090e-05 6.2443e+01]
[7.7730e-12 3.3600e-01 3.1500e-01 1.7480e-05 4.3960e-0

[1.5630e-12 3.5800e-01 3.5000e-01 8.4230e-06 2.3890e-05 6.1341e+01]
[3.2020e-12 3.3800e-01 3.3200e-01 8.4130e-06 2.5040e-05 6.1197e+01]
[1.6980e-12 3.5300e-01 3.4800e-01 8.7840e-06 2.5430e-05 6.1244e+01]
[2.4480e-12 3.4500e-01 3.4000e-01 8.4940e-06 2.4730e-05 6.1318e+01]
[2.707e-12 3.430e-01 3.360e-01 8.409e-06 2.340e-05 6.113e+01]
[2.0090e-12 3.5100e-01 3.4300e-01 8.4820e-06 2.4890e-05 6.1207e+01]
[2.0620e-12 3.5000e-01 3.4400e-01 8.9150e-06 2.4660e-05 6.1253e+01]
[1.7190e-12 3.5400e-01 3.4800e-01 8.1420e-06 2.2380e-05 6.1181e+01]
[2.8380e-12 3.4100e-01 3.3500e-01 8.5130e-06 2.6160e-05 6.1156e+01]
[1.8810e-12 3.4900e-01 3.4500e-01 8.4270e-06 2.4450e-05 6.1166e+01]
[2.1280e-12 3.4800e-01 3.4200e-01 8.6050e-06 2.4140e-05 6.0983e+01]
[1.7590e-12 3.5500e-01 3.4800e-01 8.5080e-06 2.3910e-05 6.1139e+01]
[1.9490e-12 3.5100e-01 3.4400e-01 9.0310e-06 2.7260e-05 6.1067e+01]
[2.3380e-12 3.4700e-01 3.3900e-01 8.7220e-06 2.5190e-05 6.1077e+01]
[2.9690e-12 3.4400e-01 3.3400e-01 8.5780e-06 2.6300e-0

[1.3060e-11 3.1000e-01 2.9500e-01 1.5300e-05 4.4000e-05 6.1346e+01]
[5.1490e-12 3.3300e-01 3.2300e-01 1.4540e-05 4.0480e-05 6.1836e+01]
[3.0360e-12 3.4900e-01 3.4300e-01 1.5170e-05 3.6980e-05 6.2406e+01]
[2.599e-13 4.100e-01 4.040e-01 1.311e-05 2.988e-05 6.252e+01]
[4.3560e-12 3.4800e-01 3.4500e-01 1.4980e-05 3.7400e-05 6.3913e+01]
[1.7920e-11 3.0500e-01 2.8600e-01 1.6590e-05 4.7230e-05 6.1601e+01]
[6.1020e-12 3.4200e-01 3.2600e-01 1.5410e-05 4.1870e-05 6.3228e+01]
[2.3060e-12 3.6500e-01 3.5900e-01 1.4470e-05 3.4930e-05 6.3242e+01]
[2.7620e-12 3.6600e-01 3.4100e-01 1.5240e-05 3.9380e-05 6.2442e+01]
[3.2380e-13 4.0500e-01 3.9700e-01 1.3700e-05 3.2680e-05 6.2595e+01]
[1.3170e-11 3.0800e-01 3.0300e-01 6.8460e-06 1.8790e-05 6.1212e+01]
[4.5990e-12 3.3400e-01 3.1900e-01 7.2870e-06 2.2820e-05 6.0783e+01]
[1.1950e-12 3.7500e-01 3.7200e-01 6.8310e-06 1.9130e-05 6.3068e+01]
[1.1400e-12 3.8100e-01 3.5900e-01 6.8220e-06 1.9830e-05 6.1118e+01]
[6.5840e-13 3.8300e-01 3.7800e-01 5.8800e-06 1.3890e-0

[1.0670e-11 3.1300e-01 3.0900e-01 6.3140e-06 1.7440e-05 6.1875e+01]
[3.4880e-12 3.3800e-01 3.3100e-01 6.2700e-06 1.5510e-05 6.1161e+01]
[4.6470e-12 3.3500e-01 3.2500e-01 6.0830e-06 1.7190e-05 6.0934e+01]
[2.3740e-12 3.5200e-01 3.3900e-01 6.1050e-06 1.7150e-05 6.0881e+01]
[2.1370e-12 3.5000e-01 3.4600e-01 6.3410e-06 1.6720e-05 6.1613e+01]
[2.4520e-12 3.4700e-01 3.3800e-01 5.9310e-06 1.4920e-05 6.0794e+01]
[1.242e-12 3.680e-01 3.620e-01 6.022e-06 1.511e-05 6.150e+01]
[3.6840e-12 3.4200e-01 3.2800e-01 6.6040e-06 1.9000e-05 6.0797e+01]
[1.4060e-12 3.6400e-01 3.5500e-01 6.1480e-06 1.5580e-05 6.1008e+01]
[1.4760e-12 3.5500e-01 3.5200e-01 6.1810e-06 1.8420e-05 6.0809e+01]
[1.4520e-12 3.6700e-01 3.5100e-01 6.1720e-06 1.7450e-05 6.0803e+01]
[3.3200e-12 3.4500e-01 3.3500e-01 6.0310e-06 1.6520e-05 6.0947e+01]
[1.2200e-12 3.6600e-01 3.6100e-01 5.7300e-06 1.4460e-05 6.0964e+01]
[3.0860e-12 3.3700e-01 3.3000e-01 6.3250e-06 1.8050e-05 6.0774e+01]
[1.3460e-12 3.5900e-01 3.5300e-01 6.0530e-06 1.6150e-0

[9.9090e-12 3.1400e-01 3.0400e-01 2.1940e-05 5.4190e-05 6.2089e+01]
[4.4950e-12 3.3500e-01 3.2500e-01 2.2490e-05 5.4110e-05 6.2306e+01]
[7.1720e-12 3.2400e-01 3.1400e-01 2.0480e-05 5.0440e-05 6.2238e+01]
[4.257e-12 3.400e-01 3.340e-01 1.975e-05 4.779e-05 6.279e+01]
[7.1840e-12 3.2300e-01 3.1600e-01 2.0650e-05 5.1650e-05 6.2893e+01]
[5.5790e-12 3.3300e-01 3.2200e-01 1.9810e-05 4.8820e-05 6.2713e+01]
[7.9050e-12 3.2700e-01 3.1200e-01 2.0230e-05 5.0270e-05 6.2169e+01]
[7.6870e-12 3.2000e-01 3.1100e-01 2.1350e-05 5.2750e-05 6.2079e+01]
[3.0140e-12 3.5200e-01 3.3700e-01 2.0490e-05 4.9870e-05 6.2641e+01]
[4.025e-12 3.410e-01 3.270e-01 1.926e-05 4.796e-05 6.214e+01]
[2.5850e-12 3.4800e-01 3.3900e-01 2.0000e-05 4.7730e-05 6.2005e+01]
[8.293e-12 3.220e-01 3.110e-01 2.080e-05 5.228e-05 6.227e+01]
[7.1550e-13 3.8300e-01 3.7300e-01 1.7860e-05 3.9910e-05 6.2158e+01]
[1.9940e-12 3.6400e-01 3.4500e-01 1.9070e-05 4.4700e-05 6.1982e+01]
[6.1200e-12 3.3400e-01 3.1400e-01 1.9860e-05 4.9650e-05 6.1967e+01

# LER_20201102_testset_V04

In [17]:
# file_path version : 2020_LER_20201008_V006.xlsx

print(num_total)

seed = 124
np.random.seed(seed)

file_path = '2020_LER_20201102_testset_V04.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

idx = 0
add = 0
for i in range(num_of_cycle):
    add = num_in_cycle[i]
            
    X_per_cycle[i] = X_all[idx:idx+1]
    Y_per_cycle[i] = np.mean(Y_all[idx:idx+add], axis=0)
            
    temp = X_per_cycle[i].reshape(1, num_input-2)
            
    X_all[idx:idx+add] = np.repeat(temp, add, axis=0)
    idx += add
    
# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")

data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)

data_a = []
data_b = []
data_a_X_all = []
data_a_Y_all = []
data_b_X_all = []
data_b_Y_all = []
data_a_Y_cycle = []
data_b_Y_cycle = []

# split data

for i in range(num_of_cycle):
    
    choice = [i for i in range(250)]
    mask_a = np.random.choice(choice, 125, replace=False)
    
    # complement using set
    choice = set(choice)
    mask_a = set(mask_a)  
    mask_b = choice.difference(mask_a)
    mask_b = list(mask_b)
    mask_a = list(mask_a)
#     print(mask_a)
#     print(mask_b)
    
    target_X = X_all[250*i:250*(i+1)]
    target_Y = Y_all[250*i:250*(i+1)]
#     print(target_X.shape, target_Y.shape)

    data_a_X_all.extend(target_X[mask_a])
    data_a_Y_all.extend(target_Y[mask_a])
    
    data_a_Y_cycle.append(np.mean(target_Y[mask_a], axis=0))
    
    data_b_X_all.extend(target_X[mask_b])
    data_b_Y_all.extend(target_Y[mask_b])
    
    data_b_Y_cycle.append(np.mean(target_Y[mask_b], axis=0))

data_a_X_all = np.array(data_a_X_all)
data_a_Y_all = np.array(data_a_Y_all)
data_a_Y_cyclce = np.array(data_a_Y_cycle)

data_b_X_all = np.array(data_b_X_all)
data_b_Y_all = np.array(data_b_Y_all)
data_b_Y_cyclce = np.array(data_b_Y_cycle)
    
data_a.append(data_a_X_all)
data_a.append(data_a_Y_all)
data_a.append(X_per_cycle)
data_a.append(data_a_Y_cycle)

data_b.append(data_b_X_all)
data_b.append(data_b_Y_all)
data_b.append(X_per_cycle)
data_b.append(data_b_Y_cycle)

# data save

data = np.array(data)

name = '_seed_{}'.format(seed)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0])))  
np.save('./'+file_path, data)

data_a = np.array(data_a)

print("data : data_a")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_a[0]), len(data_a[0][0]), len(data_a[2]), len(data_a[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_a[1]), len(data_a[1][0]), len(data_a[3]), len(data_a[3][0])))  
np.save('./'+file_path+'_data_a'+name, data_a)

data_b = np.array(data_b)

print("data : data_b")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_b[0]), len(data_b[0][0]), len(data_b[2]), len(data_b[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_b[1]), len(data_b[1][0]), len(data_b[3]), len(data_b[3][0])))  
np.save('./'+file_path+'_data_b'+name, data_b)

2500
          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
2495       NaN        NaN         NaN
2496       NaN        NaN         NaN
2497       NaN        NaN         NaN
2498       NaN        NaN         NaN
2499       NaN        NaN         NaN

[2500 rows x 3 columns]
============ Data load =============
X data shape:  (2500, 5) X per cycle data shape: (10, 5)
Y data shape:  (2500, 6) Y per cycle data shape: (10, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (2500,5) X per cycle data shape : (10,5)
[saved] Y data : (2500,6) Y per cycle data shape : (10,6)
data : data_a
[saved] X data : (1250,5) X per cycle data shape : (10,5)
[saved] Y data : (1250,6) Y per cycle data shape : (10,6)
data : data_b
[saved] X da

In [22]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

total_sum = 0
for i in range(Y_all.shape[0]):
    print(i, Y_all[i])
    
# for i in range(Y_all.shape[0]):
#     if 
#     print(Y_all[i])

(2500, 5) (2500, 6) (10, 5) (10, 6)
0 [1.9580e-12 4.2420e-05 1.6180e-05 3.4300e-01 3.5300e-01 6.1501e+01]
1 [6.1350e-13 3.6300e-05 1.4500e-05 3.7000e-01 3.7700e-01 6.0804e+01]
2 [7.437e-13 3.182e-05 1.391e-05 3.700e-01 3.740e-01 6.120e+01]
3 [1.4790e-12 4.1000e-05 1.5990e-05 3.5200e-01 3.5800e-01 6.1429e+01]
4 [1.9260e-12 4.3930e-05 1.7650e-05 3.4400e-01 3.5300e-01 6.1569e+01]
5 [1.1990e-12 3.6920e-05 1.4540e-05 3.5600e-01 3.6300e-01 6.1272e+01]
6 [1.6050e-12 3.6600e-05 1.3990e-05 3.4900e-01 3.5500e-01 6.1348e+01]
7 [1.5290e-12 4.0250e-05 1.5810e-05 3.5000e-01 3.5700e-01 6.1406e+01]
8 [2.2580e-12 3.9540e-05 1.6170e-05 3.4200e-01 3.5100e-01 6.1827e+01]
9 [1.9230e-12 4.2050e-05 1.5870e-05 3.4500e-01 3.5300e-01 6.1627e+01]
10 [1.1800e-12 4.0380e-05 1.7000e-05 3.5500e-01 3.6300e-01 6.1232e+01]
11 [2.461e-12 4.101e-05 1.478e-05 3.390e-01 3.490e-01 6.173e+01]
12 [3.8970e-12 4.2100e-05 1.6900e-05 3.3100e-01 3.4100e-01 6.2348e+01]
13 [2.6110e-12 4.3680e-05 1.6380e-05 3.3700e-01 3.4800e-01 6.17

318 [2.527e-12 4.379e-05 1.729e-05 3.390e-01 3.480e-01 6.191e+01]
319 [3.0160e-12 4.3160e-05 1.6930e-05 3.3600e-01 3.4500e-01 6.2043e+01]
320 [1.9840e-12 4.3440e-05 1.6510e-05 3.4300e-01 3.5100e-01 6.1493e+01]
321 [1.2240e-12 3.6950e-05 1.4650e-05 3.5500e-01 3.6200e-01 6.1158e+01]
322 [2.373e-12 4.155e-05 1.663e-05 3.400e-01 3.480e-01 6.176e+01]
323 [2.3390e-12 4.3210e-05 1.6930e-05 3.4100e-01 3.4900e-01 6.1819e+01]
324 [1.0500e-12 3.6240e-05 1.4160e-05 3.6000e-01 3.6700e-01 6.1318e+01]
325 [2.1630e-12 4.2030e-05 1.6040e-05 3.4100e-01 3.5000e-01 6.1555e+01]
326 [2.9470e-12 4.1920e-05 1.7300e-05 3.3700e-01 3.4500e-01 6.2104e+01]
327 [8.4960e-13 3.7790e-05 1.4710e-05 3.6300e-01 3.7000e-01 6.1045e+01]
328 [1.3940e-12 3.8300e-05 1.6050e-05 3.5300e-01 3.5900e-01 6.1456e+01]
329 [8.2180e-13 3.5030e-05 1.4760e-05 3.6500e-01 3.7200e-01 6.1042e+01]
330 [1.7040e-12 3.9100e-05 1.6220e-05 3.4800e-01 3.5500e-01 6.1574e+01]
331 [3.3710e-12 4.3390e-05 1.7370e-05 3.3400e-01 3.4400e-01 6.2226e+01]
332 

700 [2.4290e-12 4.3130e-05 1.6850e-05 3.3900e-01 3.4700e-01 6.1727e+01]
701 [2.4370e-12 4.3920e-05 1.7530e-05 3.3900e-01 3.4800e-01 6.1732e+01]
702 [2.209e-12 4.340e-05 1.681e-05 3.410e-01 3.490e-01 6.168e+01]
703 [1.7300e-12 4.1300e-05 1.6560e-05 3.4700e-01 3.5400e-01 6.1509e+01]
704 [2.5750e-12 4.1180e-05 1.6450e-05 3.3900e-01 3.4700e-01 6.1872e+01]
705 [2.0690e-12 4.0890e-05 1.7040e-05 3.4400e-01 3.5100e-01 6.1857e+01]
706 [4.251e-12 4.354e-05 1.762e-05 3.290e-01 3.390e-01 6.243e+01]
707 [2.5460e-12 4.3140e-05 1.7530e-05 3.4000e-01 3.4700e-01 6.1862e+01]
708 [1.3320e-12 3.7500e-05 1.4980e-05 3.5200e-01 3.6100e-01 6.1179e+01]
709 [5.0820e-12 4.3540e-05 1.8370e-05 3.2600e-01 3.3800e-01 6.2832e+01]
710 [1.967e-12 4.293e-05 1.684e-05 3.450e-01 3.520e-01 6.172e+01]
711 [2.510e-12 4.149e-05 1.642e-05 3.390e-01 3.490e-01 6.184e+01]
712 [1.711e-12 4.111e-05 1.676e-05 3.480e-01 3.550e-01 6.157e+01]
713 [1.9590e-12 4.0350e-05 1.6070e-05 3.4400e-01 3.5200e-01 6.1586e+01]
714 [1.940e-12 4.098e-

1041 [2.0720e-12 4.1510e-05 1.6260e-05 3.4200e-01 3.5100e-01 6.1524e+01]
1042 [8.790e-13 3.766e-05 1.443e-05 3.620e-01 3.680e-01 6.094e+01]
1043 [2.7430e-12 4.2090e-05 1.7230e-05 3.4000e-01 3.4700e-01 6.2173e+01]
1044 [1.8250e-12 3.9480e-05 1.5770e-05 3.4600e-01 3.5400e-01 6.1573e+01]
1045 [2.435e-12 4.145e-05 1.720e-05 3.420e-01 3.490e-01 6.204e+01]
1046 [3.0110e-12 4.1090e-05 1.7210e-05 3.3900e-01 3.4700e-01 6.2398e+01]
1047 [2.2250e-12 4.2570e-05 1.4910e-05 3.4100e-01 3.4900e-01 6.1625e+01]
1048 [1.5380e-12 4.0800e-05 1.6200e-05 3.4800e-01 3.5600e-01 6.1192e+01]
1049 [2.0920e-12 3.9420e-05 1.5720e-05 3.4500e-01 3.5200e-01 6.1912e+01]
1050 [6.1180e-12 4.4710e-05 1.8150e-05 3.2300e-01 3.3500e-01 6.3138e+01]
1051 [9.7910e-13 3.9060e-05 1.5240e-05 3.5900e-01 3.6600e-01 6.1026e+01]
1052 [1.3140e-12 3.7020e-05 1.4500e-05 3.5300e-01 3.6000e-01 6.1296e+01]
1053 [1.3730e-12 4.2200e-05 1.6310e-05 3.5200e-01 3.6000e-01 6.1429e+01]
1054 [1.8070e-12 4.0780e-05 1.5280e-05 3.4600e-01 3.5300e-01 6.

1461 [2.0400e-12 4.0800e-05 1.6200e-05 3.4400e-01 3.5100e-01 6.1695e+01]
1462 [2.7670e-12 3.8830e-05 1.6020e-05 3.3800e-01 3.4700e-01 6.1894e+01]
1463 [2.1540e-12 3.9130e-05 1.5200e-05 3.4200e-01 3.4900e-01 6.1579e+01]
1464 [3.182e-12 4.049e-05 1.598e-05 3.340e-01 3.430e-01 6.201e+01]
1465 [1.8290e-12 4.2200e-05 1.6730e-05 3.4600e-01 3.5400e-01 6.1567e+01]
1466 [1.8100e-12 3.3940e-05 1.3900e-05 3.4600e-01 3.5300e-01 6.1534e+01]
1467 [3.6210e-12 4.2770e-05 1.7320e-05 3.3200e-01 3.4400e-01 6.2387e+01]
1468 [2.6350e-12 4.1890e-05 1.7290e-05 3.4100e-01 3.4800e-01 6.2232e+01]
1469 [8.2640e-12 4.5900e-05 1.8270e-05 3.1800e-01 3.3000e-01 6.3635e+01]
1470 [1.6840e-12 3.8440e-05 1.4710e-05 3.4700e-01 3.5600e-01 6.1436e+01]
1471 [1.6500e-12 4.2260e-05 1.6450e-05 3.4700e-01 3.5600e-01 6.1351e+01]
1472 [3.2980e-12 4.4460e-05 1.7160e-05 3.3500e-01 3.4400e-01 6.2272e+01]
1473 [3.0820e-12 4.1350e-05 1.6330e-05 3.3600e-01 3.4600e-01 6.2134e+01]
1474 [5.7070e-12 4.2640e-05 1.8380e-05 3.2600e-01 3.3800e

1871 [1.2000e-12 4.0630e-05 1.5660e-05 3.5600e-01 3.6200e-01 6.1255e+01]
1872 [2.7100e-12 3.8660e-05 1.5180e-05 3.3800e-01 3.4600e-01 6.1875e+01]
1873 [5.2660e-12 4.3020e-05 1.7770e-05 3.2600e-01 3.3700e-01 6.2985e+01]
1874 [2.3890e-12 4.3500e-05 1.7250e-05 3.4100e-01 3.4900e-01 6.1779e+01]
1875 [7.2200e-13 3.2630e-05 1.3760e-05 3.6900e-01 3.7400e-01 6.1187e+01]
1876 [2.2430e-12 4.2440e-05 1.5430e-05 3.4200e-01 3.5000e-01 6.1745e+01]
1877 [2.8310e-12 4.4430e-05 1.7620e-05 3.3600e-01 3.4600e-01 6.1925e+01]
1878 [1.4800e-12 4.1530e-05 1.5300e-05 3.4900e-01 3.5700e-01 6.1246e+01]
1879 [2.5830e-12 4.2210e-05 1.7100e-05 3.3800e-01 3.4700e-01 6.1784e+01]
1880 [1.2040e-12 3.6680e-05 1.5440e-05 3.5600e-01 3.6200e-01 6.1341e+01]
1881 [1.5260e-12 3.9490e-05 1.5460e-05 3.5000e-01 3.5800e-01 6.1384e+01]
1882 [2.7510e-12 4.2610e-05 1.8020e-05 3.3900e-01 3.4700e-01 6.2043e+01]
1883 [2.9110e-12 4.2490e-05 1.6020e-05 3.3600e-01 3.4500e-01 6.1915e+01]
1884 [1.7840e-12 3.9740e-05 1.6320e-05 3.4600e-01 3

2314 [2.2240e-12 4.3170e-05 1.6660e-05 3.4100e-01 3.4800e-01 6.1634e+01]
2315 [2.3250e-12 4.3380e-05 1.6850e-05 3.4000e-01 3.4800e-01 6.1671e+01]
2316 [2.3290e-12 4.3130e-05 1.6940e-05 3.4000e-01 3.4800e-01 6.1705e+01]
2317 [2.5660e-12 4.3390e-05 1.6990e-05 3.3800e-01 3.4600e-01 6.1779e+01]
2318 [2.1560e-12 4.2960e-05 1.6750e-05 3.4200e-01 3.4900e-01 6.1611e+01]
2319 [2.2050e-12 4.2930e-05 1.6550e-05 3.4100e-01 3.4900e-01 6.1615e+01]
2320 [2.0910e-12 4.2680e-05 1.6550e-05 3.4200e-01 3.5000e-01 6.1582e+01]
2321 [2.1520e-12 4.3690e-05 1.6480e-05 3.4100e-01 3.4900e-01 6.1572e+01]
2322 [1.9750e-12 4.3010e-05 1.6500e-05 3.4400e-01 3.5100e-01 6.1586e+01]
2323 [2.3620e-12 4.3050e-05 1.6780e-05 3.4000e-01 3.4800e-01 6.1676e+01]
2324 [2.1400e-12 4.3820e-05 1.6750e-05 3.4200e-01 3.4900e-01 6.1606e+01]
2325 [2.2350e-12 4.2550e-05 1.6420e-05 3.4100e-01 3.4800e-01 6.1614e+01]
2326 [2.0450e-12 4.2460e-05 1.6490e-05 3.4300e-01 3.5000e-01 6.1597e+01]
2327 [2.3470e-12 4.3050e-05 1.6820e-05 3.4000e-01 3

# 검증의 시간

In [92]:
# 혼돈의 시작
# 앙 기모띠 >_<
file_path1 = '2020_LER_20201021_testset.xlsx'
file_path2 = '2020_LER_20200804_V006.xlsx'

file1 = np.load(file_path1+'.npy', allow_pickle=True)
file2 = np.load(file_path2+'.npy', allow_pickle=True)

X_all_1, Y_all_1, X_per_cycle_1, Y_per_cycle_1 = file1[0], file1[1], file1[2], file1[3]
X_all_2, Y_all_2, X_per_cycle_2, Y_per_cycle_2 = file2[0], file2[1], file2[2], file2[3]

X_per_cycle_2_test = X_per_cycle_2[85:]

new_factor = 2
old_factor = 7

print(X_per_cycle_1)
print(X_per_cycle_2_test.shape)

print(X_per_cycle_1[new_factor])
print(X_per_cycle_2_test[old_factor])

 


# for i in range(5):
#     print(X_per_cycle_1[i])
    
# # print(X_per_cycle_2_test)
# for j in range(15):
#     print(X_per_cycle_2_test[i])

[[  0.62484672  17.00743225 173.85718891   1.           0.        ]
 [  0.55654727  80.91905766  80.56986979   1.           0.        ]
 [  0.50466944  48.62398816  67.99380535   1.           0.        ]
 [  0.56045148  25.42868151  69.39740975   1.           0.        ]
 [  0.63730798  69.14860261  65.33037708   1.           0.        ]]
(15, 5)
[ 0.50466944 48.62398816 67.99380535  1.          0.        ]
[ 0.50466944 48.62398816 67.99380535  1.          0.        ]


# constant currrent Test_set V00

In [84]:
# file_path version : 2020_LER_20201008_V006.xlsx

# seed 고정
seed = 0
np.random.seed(seed)

print(num_total)
file_path = 'constant current Test set_V00.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Test_set', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

idx = 0
add = 0
for i in range(num_of_cycle):
    add = num_in_cycle[i]
            
    X_per_cycle[i] = X_all[idx:idx+1]
    Y_per_cycle[i] = np.mean(Y_all[idx:idx+add], axis=0)
            
    temp = X_per_cycle[i].reshape(1, num_input-2)
            
    X_all[idx:idx+add] = np.repeat(temp, add, axis=0)
    idx += add
    
# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")

data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)

data_a = []
data_b = []
data_a_X_all = []
data_a_Y_all = []
data_b_X_all = []
data_b_Y_all = []
data_a_Y_cycle = []
data_b_Y_cycle = []

# split data

for i in range(num_of_cycle):
    
    choice = [i for i in range(250)]
    mask_a = np.random.choice(choice, 125, replace=False)
    
    # complement using set
    choice = set(choice)
    mask_a = set(mask_a)  
    mask_b = choice.difference(mask_a)
    mask_b = list(mask_b)
    mask_a = list(mask_a)
#     print(mask_a)
#     print(mask_b)
    
    target_X = X_all[250*i:250*(i+1)]
    target_Y = Y_all[250*i:250*(i+1)]
#     print(target_X.shape, target_Y.shape)

    data_a_X_all.extend(target_X[mask_a])
    data_a_Y_all.extend(target_Y[mask_a])
    
    data_a_Y_cycle.append(np.mean(target_Y[mask_a], axis=0))
    
    data_b_X_all.extend(target_X[mask_b])
    data_b_Y_all.extend(target_Y[mask_b])
    
    data_b_Y_cycle.append(np.mean(target_Y[mask_b], axis=0))


data_a.append(data_a_X_all)
data_a.append(data_a_Y_all)
data_a.append(X_per_cycle)
data_a.append(data_a_Y_cycle)

data_b.append(data_b_X_all)
data_b.append(data_b_Y_all)
data_b.append(X_per_cycle)
data_b.append(data_b_Y_cycle)

# data save

data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0])))  
np.save('./'+file_path, data)

data_a = np.array(data_a)

print("data : data_a")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_a[0]), len(data_a[0][0]), len(data_a[2]), len(data_a[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_a[1]), len(data_a[1][0]), len(data_a[3]), len(data_a[3][0])))  
np.save('./'+file_path+'_data_a', data_a)

data_b = np.array(data_b)

print("data : data_b")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data_b[0]), len(data_b[0][0]), len(data_b[2]), len(data_b[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data_b[1]), len(data_b[1][0]), len(data_b[3]), len(data_b[3][0])))  
np.save('./'+file_path+'_data_b', data_b)

2500
          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
2495       NaN        NaN         NaN
2496       NaN        NaN         NaN
2497       NaN        NaN         NaN
2498       NaN        NaN         NaN
2499       NaN        NaN         NaN

[2500 rows x 3 columns]
============ Data load =============
X data shape:  (2500, 5) X per cycle data shape: (10, 5)
Y data shape:  (2500, 6) Y per cycle data shape: (10, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (2500,5) X per cycle data shape : (10,5)
[saved] Y data : (2500,6) Y per cycle data shape : (10,6)
data : data_a
[saved] X data : (1250,5) X per cycle data shape : (10,5)
[saved] Y data : (1250,6) Y per cycle data shape : (10,6)
data : data_b
[saved] X da

In [31]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)


#print(Y_all[])

for i in range(len(Y_all)):
    print(Y_all[i])
#print(Y_all[1306:1556])

# total_sum = 0
# for i in [250]*10:
#     total_sum += i
#     print(total_sum)
    
# for i in range(Y_all.shape[0]):
#     if 
#     print(Y_all[i])

(2500, 5) (2500, 6) (10, 5) (10, 6)
[1.95800000e-12 4.24200000e-05 1.61800000e-05 3.43000000e-01
 3.53000000e-01 6.29076455e+01]
[6.13500000e-13 3.63000000e-05 1.45000000e-05 3.70000000e-01
 3.77000000e-01 6.21155735e+01]
[7.43700000e-13 3.18200000e-05 1.39100000e-05 3.70000000e-01
 3.74000000e-01 6.30020099e+01]
[1.47900000e-12 4.10000000e-05 1.59900000e-05 3.52000000e-01
 3.58000000e-01 6.31478319e+01]
[1.92600000e-12 4.39300000e-05 1.76500000e-05 3.44000000e-01
 3.53000000e-01 6.30086444e+01]
[1.19900000e-12 3.69200000e-05 1.45400000e-05 3.56000000e-01
 3.63000000e-01 6.28370344e+01]
[1.60500000e-12 3.66000000e-05 1.39900000e-05 3.49000000e-01
 3.55000000e-01 6.30106175e+01]
[1.52900000e-12 4.02500000e-05 1.58100000e-05 3.50000000e-01
 3.57000000e-01 6.29515477e+01]
[2.25800000e-12 3.95400000e-05 1.61700000e-05 3.42000000e-01
 3.51000000e-01 6.34462298e+01]
[1.92300000e-12 4.20500000e-05 1.58700000e-05 3.45000000e-01
 3.53000000e-01 6.31844852e+01]
[1.18000000e-12 4.03800000e-05 1.7

 3.64000000e-01 6.23756232e+01]
[2.73300000e-12 4.06900000e-05 1.69400000e-05 3.38000000e-01
 3.47000000e-01 6.36844679e+01]
[1.85900000e-12 4.07400000e-05 1.57900000e-05 3.45000000e-01
 3.53000000e-01 6.30143455e+01]
[2.24000000e-12 4.10600000e-05 1.64000000e-05 3.42000000e-01
 3.50000000e-01 6.34052239e+01]
[1.8590000e-12 3.8870000e-05 1.5610000e-05 3.4700000e-01 3.5400000e-01
 6.3380708e+01]
[1.58300000e-12 3.94100000e-05 1.45900000e-05 3.48000000e-01
 3.56000000e-01 6.27614359e+01]
[3.91400000e-12 4.36600000e-05 1.73000000e-05 3.30000000e-01
 3.41000000e-01 6.40609632e+01]
[1.99900000e-12 4.33700000e-05 1.70100000e-05 3.43000000e-01
 3.51000000e-01 6.30119608e+01]
[3.63100000e-12 4.08000000e-05 1.65500000e-05 3.34000000e-01
 3.43000000e-01 6.44309011e+01]
[1.93400000e-12 3.84000000e-05 1.46200000e-05 3.44000000e-01
 3.53000000e-01 6.30294878e+01]
[2.11300000e-12 4.09900000e-05 1.63300000e-05 3.44000000e-01
 3.51000000e-01 6.34779128e+01]
[1.69700000e-12 3.80000000e-05 1.48500000e-0

[1.68200000e-12 4.21400000e-05 1.64300000e-05 3.47000000e-01
 3.55000000e-01 6.28801913e+01]
[2.05300000e-12 4.24400000e-05 1.64700000e-05 3.43000000e-01
 3.50000000e-01 6.31466434e+01]
[2.80800000e-12 4.40700000e-05 1.74400000e-05 3.36000000e-01
 3.46000000e-01 6.34474903e+01]
[1.96600000e-12 4.10300000e-05 1.64700000e-05 3.45000000e-01
 3.51000000e-01 6.32961437e+01]
[2.16000000e-12 4.22000000e-05 1.65600000e-05 3.42000000e-01
 3.50000000e-01 6.32195625e+01]
[2.37400000e-12 4.19900000e-05 1.66700000e-05 3.40000000e-01
 3.48000000e-01 6.33304761e+01]
[1.53000000e-12 4.16100000e-05 1.58800000e-05 3.49000000e-01
 3.56000000e-01 6.27743859e+01]
[2.04000000e-12 4.14200000e-05 1.67100000e-05 3.43000000e-01
 3.51000000e-01 6.31144942e+01]
[2.38200000e-12 4.16500000e-05 1.62100000e-05 3.40000000e-01
 3.47000000e-01 6.33477666e+01]
[2.15300000e-12 4.26500000e-05 1.63000000e-05 3.42000000e-01
 3.49000000e-01 6.32030441e+01]
[2.19200000e-12 4.30800000e-05 1.72300000e-05 3.42000000e-01
 3.490000

# 2020_LER_20201008_V008_logIoff

In [40]:
# file_path version : 2020_LER_20201008_V008.xlsx

file_path = '2020_LER_20201008_V008.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
#     print(i*num_in_cycle, i*num_in_cycle+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle[i])


# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))


# Y_all
for i in range(num_of_cycle):
#     print(i*num_in_cycle,(i+1)*num_in_cycle)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all[i*num_in_cycle])

# log Ioff
print(Y_all[:,0])

Y_all[:,0] = np.log(Y_all[:,0])
print(Y_all[:,0])

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./log_ioff'+file_path, data)

          amp.     corr.x      corr.y
0     0.193163  61.002273  110.617897
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
6345       NaN        NaN         NaN
6346       NaN        NaN         NaN
6347       NaN        NaN         NaN
6348       NaN        NaN         NaN
6349       NaN        NaN         NaN

[6350 rows x 3 columns]
[2.394e-12 2.043e-12 2.537e-12 ... 1.437e-12 2.380e-12 2.148e-12]
[-26.75805551 -26.9166018  -26.70003884 ... -27.26846351 -26.76392063
 -26.86648394]
============ Data load =============
X data shape:  (6350, 5) X per cycle data shape: (127, 5)
Y data shape:  (6350, 6) Y per cycle data shape: (127, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


# 검증의 시간

In [45]:
# 검증의 시간

data = np.load('log_ioff'+file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(len(Y_all)):
    print(Y_all[i])

(6350, 5) (6350, 6) (127, 5) (127, 6)
[-2.67580555e+01  4.31100000e-05  1.67900000e-05  3.39000000e-01
  3.47000000e-01  6.16950000e+01]
[-2.69166018e+01  4.26200000e-05  1.66200000e-05  3.43000000e-01
  3.50000000e-01  6.15530000e+01]
[-2.67000388e+01  4.39800000e-05  1.69700000e-05  3.38000000e-01
  3.46000000e-01  6.17460000e+01]
[-2.67710891e+01  4.36100000e-05  1.69900000e-05  3.40000000e-01
  3.47000000e-01  6.16980000e+01]
[-2.6861376e+01  4.2830000e-05  1.6680000e-05  3.4100000e-01
  3.4900000e-01  6.1591000e+01]
[-2.69896937e+01  4.25100000e-05  1.59300000e-05  3.44000000e-01
  3.52000000e-01  6.15200000e+01]
[-2.69719479e+01  4.32400000e-05  1.65500000e-05  3.44000000e-01
  3.51000000e-01  6.15360000e+01]
[-2.6669757e+01  4.3160000e-05  1.7150000e-05  3.3800000e-01
  3.4600000e-01  6.1811000e+01]
[-2.68905134e+01  4.26600000e-05  1.66200000e-05  3.42000000e-01
  3.50000000e-01  6.15590000e+01]
[-2.67447773e+01  4.38500000e-05  1.72700000e-05  3.39000000e-01
  3.47000000e-01  

[-2.69519729e+01  4.25300000e-05  1.63500000e-05  3.43000000e-01
  3.50000000e-01  6.15450000e+01]
[-2.68362203e+01  4.25100000e-05  1.64800000e-05  3.41000000e-01
  3.48000000e-01  6.16130000e+01]
[-2.68299168e+01  4.29100000e-05  1.63100000e-05  3.41000000e-01
  3.49000000e-01  6.16200000e+01]
[-2.67761803e+01  4.34700000e-05  1.68200000e-05  3.40000000e-01
  3.48000000e-01  6.16570000e+01]
[-2.69509592e+01  4.32000000e-05  1.65700000e-05  3.43000000e-01
  3.51000000e-01  6.15170000e+01]
[-2.69358759e+01  4.26700000e-05  1.65200000e-05  3.43000000e-01
  3.50000000e-01  6.15650000e+01]
[-2.67382037e+01  4.34200000e-05  1.69500000e-05  3.39000000e-01
  3.47000000e-01  6.17520000e+01]
[-2.67668661e+01  4.32800000e-05  1.67700000e-05  3.40000000e-01
  3.47000000e-01  6.16990000e+01]
[-2.67386133e+01  4.35700000e-05  1.68000000e-05  3.39000000e-01
  3.47000000e-01  6.17470000e+01]
[-2.69565472e+01  4.20500000e-05  1.60700000e-05  3.44000000e-01
  3.51000000e-01  6.15270000e+01]
[-2.684620

  3.45000000e-01  6.19060000e+01]
[-2.68200909e+01  4.38100000e-05  1.68100000e-05  3.40000000e-01
  3.49000000e-01  6.16330000e+01]
[-2.67881617e+01  4.28500000e-05  1.67200000e-05  3.40000000e-01
  3.48000000e-01  6.16830000e+01]
[-2.68772493e+01  4.24800000e-05  1.65700000e-05  3.42000000e-01
  3.49000000e-01  6.16410000e+01]
[-2.68143297e+01  4.33700000e-05  1.70200000e-05  3.41000000e-01
  3.49000000e-01  6.16810000e+01]
[-2.70532848e+01  4.14200000e-05  1.64100000e-05  3.46000000e-01
  3.53000000e-01  6.14500000e+01]
[-2.67791622e+01  4.28500000e-05  1.68900000e-05  3.40000000e-01
  3.48000000e-01  6.16730000e+01]
[-2.68086016e+01  4.34900000e-05  1.69100000e-05  3.40000000e-01
  3.48000000e-01  6.16380000e+01]
[-2.69693646e+01  4.19500000e-05  1.64300000e-05  3.44000000e-01
  3.51000000e-01  6.14760000e+01]
[-2.6933384e+01  4.3720000e-05  1.6900000e-05  3.4300000e-01
  3.5100000e-01  6.1567000e+01]
[-2.67942066e+01  4.27200000e-05  1.65500000e-05  3.40000000e-01
  3.48000000e-01

[-2.67341171e+01  4.37200000e-05  1.73200000e-05  3.40000000e-01
  3.48000000e-01  6.18820000e+01]
[-2.67994172e+01  4.39100000e-05  1.76500000e-05  3.42000000e-01
  3.50000000e-01  6.18910000e+01]
[-2.6527753e+01  4.3980000e-05  1.7020000e-05  3.3400000e-01
  3.4500000e-01  6.1899000e+01]
[-2.82857171e+01  2.96400000e-05  1.24100000e-05  3.77000000e-01
  3.80000000e-01  6.10590000e+01]
[-2.74412275e+01  4.04600000e-05  1.57100000e-05  3.55000000e-01
  3.63000000e-01  6.11750000e+01]
[-2.74272643e+01  4.01000000e-05  1.56200000e-05  3.55000000e-01
  3.63000000e-01  6.12980000e+01]
[-2.68402936e+01  4.25300000e-05  1.67000000e-05  3.42000000e-01
  3.49000000e-01  6.17520000e+01]
[-2.5932657e+01  4.4190000e-05  1.7690000e-05  3.2400000e-01
  3.3700000e-01  6.2886000e+01]
[-2.63762601e+01  4.39700000e-05  1.70300000e-05  3.33000000e-01
  3.43000000e-01  6.23860000e+01]
[-2.73771544e+01  3.98900000e-05  1.51200000e-05  3.54000000e-01
  3.61000000e-01  6.13610000e+01]
[-2.6611452e+01  4.207

  3.49000000e-01  6.16750000e+01]
[-2.69249577e+01  4.29600000e-05  1.67100000e-05  3.43000000e-01
  3.50000000e-01  6.15760000e+01]
[-2.675264e+01  4.194000e-05  1.648000e-05  3.400000e-01  3.480000e-01
  6.170700e+01]
[-2.68457506e+01  4.31700000e-05  1.69900000e-05  3.42000000e-01
  3.49000000e-01  6.16780000e+01]
[-2.66886728e+01  4.25200000e-05  1.66300000e-05  3.38000000e-01
  3.46000000e-01  6.17570000e+01]
[-2.71966447e+01  4.14600000e-05  1.60200000e-05  3.49000000e-01
  3.56000000e-01  6.13300000e+01]
[-2.65638995e+01  4.32500000e-05  1.72900000e-05  3.36000000e-01
  3.44000000e-01  6.18990000e+01]
[-2.64839362e+01  4.27000000e-05  1.71400000e-05  3.35000000e-01
  3.43000000e-01  6.20680000e+01]
[-2.68725545e+01  4.27200000e-05  1.67100000e-05  3.42000000e-01
  3.49000000e-01  6.15830000e+01]
[-2.6839387e+01  4.0860000e-05  1.6400000e-05  3.4200000e-01
  3.4900000e-01  6.1684000e+01]
[-2.67095439e+01  4.34300000e-05  1.68800000e-05  3.39000000e-01
  3.47000000e-01  6.17870000

  3.41000000e-01  6.23910000e+01]
[-2.64395258e+01  4.28800000e-05  1.69200000e-05  3.33000000e-01
  3.44000000e-01  6.20370000e+01]
[-2.68335139e+01  3.96200000e-05  1.59500000e-05  3.44000000e-01
  3.50000000e-01  6.18940000e+01]
[-2.64989741e+01  4.20500000e-05  1.68500000e-05  3.35000000e-01
  3.45000000e-01  6.21050000e+01]
[-2.67443652e+01  4.38700000e-05  1.65600000e-05  3.39000000e-01
  3.47000000e-01  6.15910000e+01]
[-2.62962832e+01  4.15500000e-05  1.65800000e-05  3.33000000e-01
  3.42000000e-01  6.24650000e+01]
[-2.62611102e+01  4.24200000e-05  1.71200000e-05  3.31000000e-01
  3.40000000e-01  6.23540000e+01]
[-2.68599874e+01  4.01400000e-05  1.56500000e-05  3.43000000e-01
  3.50000000e-01  6.17710000e+01]
[-2.69097725e+01  4.18900000e-05  1.63400000e-05  3.42000000e-01
  3.52000000e-01  6.15460000e+01]
[-2.71116323e+01  4.20300000e-05  1.63800000e-05  3.48000000e-01
  3.55000000e-01  6.15550000e+01]
[-2.66241587e+01  4.08600000e-05  1.71700000e-05  3.38000000e-01
  3.460000

  3.50000000e-01  6.16520000e+01]
[-2.68962523e+01  4.15100000e-05  1.63300000e-05  3.43000000e-01
  3.51000000e-01  6.16330000e+01]
[-2.67163317e+01  4.22800000e-05  1.59300000e-05  3.39000000e-01
  3.47000000e-01  6.17910000e+01]
[-2.69933866e+01  4.20900000e-05  1.65600000e-05  3.44000000e-01
  3.52000000e-01  6.14660000e+01]
[-2.64131454e+01  4.36600000e-05  1.75700000e-05  3.34000000e-01
  3.43000000e-01  6.23380000e+01]
[-2.70382469e+01  3.85700000e-05  1.57300000e-05  3.46000000e-01
  3.53000000e-01  6.15320000e+01]
[-2.66216044e+01  4.21200000e-05  1.69000000e-05  3.38000000e-01
  3.46000000e-01  6.19350000e+01]
[-2.65217986e+01  4.28300000e-05  1.71000000e-05  3.36000000e-01
  3.44000000e-01  6.21670000e+01]
[-2.68326134e+01  4.24100000e-05  1.61900000e-05  3.41000000e-01
  3.49000000e-01  6.16410000e+01]
[-2.66964976e+01  4.37200000e-05  1.70900000e-05  3.38000000e-01
  3.47000000e-01  6.18020000e+01]
[-2.66194202e+01  4.32300000e-05  1.69700000e-05  3.37000000e-01
  3.460000

# 2021_RDFWFV_20210107

In [109]:
# file_path version : 2021_RDFWFV_20210107.xlsx

# elif file_path == '2021_RDFWFV_20210107.xlsx':
#     num_input = 6
#     one_hot = 3
#     num_output = 8
#     num_in_cycle = 250
#     num_of_cycle = [5, 5, 6]
#     num_total = 250*sum(num_of_cycle)
#     x_cols = "A:F"
#     y_cols = "G:N"
#     header = 0

print(num_total)
file_path = '2021_RDFWFV_20210107.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Sheet1', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Sheet1', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((sum(num_of_cycle), num_input)), np.zeros((sum(num_of_cycle), num_output))

# DATA_X DATA_Y preprocessing

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

for i in range(sum(num_of_cycle)):
    X_per_cycle[i] = X_all[num_in_cycle*i: num_in_cycle*i+1]
    Y_per_cycle[i] = np.mean(Y_all[num_in_cycle*i: num_in_cycle*(i+1)])

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)

print("data : data_all")
print("[saved] X data : ({},{}) X per cycle data shape : ({},{})".format(len(data[0]), len(data[0][0]), len(data[2]), len(data[2][0])))                                                                  
print("[saved] Y data : ({},{}) Y per cycle data shape : ({},{})".format(len(data[1]), len(data[1][0]), len(data[3]), len(data[3][0])))  
np.save('./'+file_path, data)

4000
============ Data load =============
X data shape:  (4000, 6) X per cycle data shape: (16, 6)
Y data shape:  (4000, 8) Y per cycle data shape: (16, 8)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
data : data_all
[saved] X data : (4000,6) X per cycle data shape : (16,6)
[saved] Y data : (4000,8) Y per cycle data shape : (16,8)


# 검증의 시간

In [110]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(len(Y_all)):
    print(Y_all[i])

(4000, 6) (4000, 8) (16, 6) (16, 8)
[4.0700e-12 3.3900e-01 3.2800e-01 2.2060e-05 5.1360e-05 6.2333e+01
 1.0340e-06 4.9050e-05]
[4.335e-12 3.380e-01 3.270e-01 2.270e-05 5.326e-05 6.238e+01 1.065e-06
 5.054e-05]
[4.2670e-12 3.3800e-01 3.2800e-01 2.2980e-05 5.2980e-05 6.2536e+01
 1.0570e-06 5.0830e-05]
[4.3920e-12 3.3800e-01 3.2700e-01 2.1990e-05 5.1700e-05 6.2396e+01
 1.0740e-06 4.9010e-05]
[4.1970e-12 3.3700e-01 3.2800e-01 2.3030e-05 5.1790e-05 6.2324e+01
 1.0520e-06 4.9950e-05]
[4.1660e-12 3.3800e-01 3.2800e-01 2.2650e-05 5.1220e-05 6.2379e+01
 1.0450e-06 4.9330e-05]
[4.3750e-12 3.3700e-01 3.2600e-01 2.5310e-05 5.6130e-05 6.2358e+01
 1.0950e-06 5.3860e-05]
[4.6100e-12 3.3600e-01 3.2600e-01 2.3580e-05 5.3220e-05 6.2423e+01
 1.1100e-06 5.1220e-05]
[4.4200e-12 3.3700e-01 3.2700e-01 2.2330e-05 5.1590e-05 6.2425e+01
 1.0820e-06 4.9320e-05]
[4.1650e-12 3.3800e-01 3.2800e-01 2.1700e-05 5.0770e-05 6.2394e+01
 1.0460e-06 4.8440e-05]
[4.2120e-12 3.3800e-01 3.2800e-01 2.3780e-05 5.2670e-05 6.2389

 6.0770e-07 1.7520e-05]
[1.9930e-12 3.5000e-01 3.4400e-01 6.7170e-06 1.9610e-05 6.1065e+01
 6.4410e-07 1.9080e-05]
[2.2030e-12 3.4800e-01 3.4100e-01 7.3220e-06 2.2070e-05 6.1109e+01
 7.1390e-07 2.1200e-05]
[2.132e-12 3.480e-01 3.420e-01 6.990e-06 2.089e-05 6.108e+01 6.882e-07
 2.017e-05]
[1.9960e-12 3.5000e-01 3.4300e-01 6.8070e-06 1.9010e-05 6.0933e+01
 6.6000e-07 1.8530e-05]
[2.0260e-12 3.4900e-01 3.4400e-01 7.2410e-06 1.9880e-05 6.1154e+01
 6.4890e-07 1.9420e-05]
[2.2640e-12 3.4800e-01 3.4100e-01 6.8110e-06 1.9100e-05 6.1118e+01
 7.0400e-07 1.8580e-05]
[2.3500e-12 3.4600e-01 3.3900e-01 7.0130e-06 2.0680e-05 6.0994e+01
 7.4050e-07 2.0000e-05]
[2.1040e-12 3.4900e-01 3.4200e-01 7.0210e-06 2.0250e-05 6.1107e+01
 6.7610e-07 1.9680e-05]
[2.1510e-12 3.4800e-01 3.4100e-01 7.0190e-06 2.0980e-05 6.1065e+01
 6.9580e-07 2.0320e-05]
[2.1340e-12 3.4800e-01 3.4200e-01 7.1680e-06 2.1020e-05 6.1104e+01
 6.9050e-07 2.0350e-05]
[2.100e-12 3.480e-01 3.430e-01 7.125e-06 1.888e-05 6.106e+01 6.705e-07
 1.

 1.7310e-07 3.1470e-05]
[1.325e-12 3.770e-01 3.760e-01 1.547e-05 3.045e-05 6.370e+01 2.547e-07
 3.035e-05]
[3.1060e-11 2.8700e-01 2.7900e-01 1.6670e-05 4.0960e-05 6.2619e+01
 2.8480e-06 3.9940e-05]
[1.1910e-12 3.6800e-01 3.6200e-01 1.5390e-05 3.3870e-05 6.1513e+01
 3.8170e-07 3.3330e-05]
[1.7260e-13 4.1900e-01 4.0600e-01 1.3990e-05 3.0050e-05 6.1116e+01
 8.2690e-08 2.8900e-05]
[6.4130e-11 2.7100e-01 2.5500e-01 1.6760e-05 4.5730e-05 6.1735e+01
 4.0880e-06 4.2930e-05]
[1.4320e-12 3.6600e-01 3.5400e-01 1.5620e-05 3.5930e-05 6.1563e+01
 4.7830e-07 3.4980e-05]
[4.6460e-11 2.8700e-01 2.6200e-01 1.6730e-05 4.4100e-05 6.1575e+01
 3.6270e-06 4.0940e-05]
[2.831e-11 2.930e-01 2.870e-01 1.489e-05 3.427e-05 6.212e+01 1.932e-06
 3.352e-05]
[6.0640e-12 3.4000e-01 3.3900e-01 1.4850e-05 3.2270e-05 6.3771e+01
 7.1170e-07 3.1690e-05]
[1.3830e-11 3.0100e-01 2.9500e-01 1.6920e-05 4.2950e-05 6.1949e+01
 2.2440e-06 4.1870e-05]
[1.7080e-11 3.1100e-01 2.8800e-01 1.5200e-05 3.6460e-05 6.1272e+01
 2.1760e-06 3.5

 4.7580e-07 1.4950e-05]
[3.7940e-12 3.4300e-01 3.3700e-01 6.2880e-06 1.7790e-05 6.2165e+01
 7.6060e-07 1.7340e-05]
[1.5510e-12 3.5700e-01 3.4900e-01 6.4590e-06 1.6960e-05 6.0852e+01
 5.4350e-07 1.6660e-05]
[5.5570e-12 3.3500e-01 3.3100e-01 6.1290e-06 1.7040e-05 6.1608e+01
 8.5150e-07 1.6590e-05]
[2.1100e-12 3.4900e-01 3.4400e-01 6.1600e-06 1.6650e-05 6.1036e+01
 6.3260e-07 1.6240e-05]
[2.7750e-12 3.4700e-01 3.3600e-01 6.1570e-06 1.6870e-05 6.0903e+01
 7.9480e-07 1.6180e-05]
[5.7570e-12 3.3100e-01 3.2400e-01 5.9490e-06 1.4640e-05 6.0998e+01
 9.7220e-07 1.4290e-05]
[1.3750e-12 3.6800e-01 3.6200e-01 5.7160e-06 1.4350e-05 6.1126e+01
 3.9670e-07 1.4040e-05]
[1.6070e-12 3.6300e-01 3.5800e-01 5.9110e-06 1.4800e-05 6.1591e+01
 4.3400e-07 1.4660e-05]
[3.7250e-12 3.4000e-01 3.2800e-01 6.2800e-06 1.7260e-05 6.0971e+01
 9.5680e-07 1.6650e-05]
[4.7330e-12 3.3500e-01 3.2000e-01 5.7830e-06 1.5150e-05 6.0805e+01
 1.1390e-06 1.4670e-05]
[6.1650e-13 3.8000e-01 3.7400e-01 6.2250e-06 1.6120e-05 6.0827e+01